## 03. Curățarea și pregătirea datelor

### Importul librariilor

In [2]:
! pip install pyodbc

In [3]:
!  pip install sqlalchemy

In [4]:
import pandas as pd
import numpy as np
import json
import os
import pyodbc
import sqlalchemy as db
import urllib

exec(open('credentials.py').read())

### Conectarea la Azure Database

In [5]:
database_name = "AID2023"
driver = '{ODBC Driver 18 for SQL Server}'
server_name = "aid-2023-srv.database.windows.net"

In [6]:
# Create the server url
server = 'tcp:{server_name},1433'.format(server_name = server_name)

#create the full connection string <- make sure to write it as it is bellow - check your credentials to be the right ones

connection_string = 'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};charset=utf8'.format(
    driver = driver,
    server=server,
    database=database_name,
    username="aid2023adminlog",
    password="@Aidr2023sql"
)                              

cnxn: pyodbc.Connection = pyodbc.connect(connection_string)
cursor: pyodbc.Cursor = cnxn.cursor() 

#execute a query with the cursor to check if the connection is maade
print("Connection is open") if cursor.execute("SELECT 1") else print(f"Connection is closed: {e}")

Connection is open


### Preluarea tabelului cu informatii despre scolile din judetul Cluj

In [7]:
#creez un query pentru import
query = 'SELECT * FROM school_details_cj_team2'
cursor.execute(query)
school_details_cj_team2 = cursor.fetchall()
school_details_cj_team2

[(11266861, '123007621', 'CASA CORPULUI DIDACTIC CLUJ', 'CLUJ', 'CLUJ-NAPOCA', 'Publică de interes naţional şi local', 'Buget', 0, 0, 100),
 (11266905, '12000012', 'C.J.R.A.E. CLUJ', 'CLUJ', 'CLUJ-NAPOCA', 'Publică de interes naţional şi local', 'Buget', 0, 0, 157),
 (11266906, '120447612', 'CSEI CJ (SC. SPEC. 1)', 'CLUJ', 'CLUJ-NAPOCA', 'Publică de interes naţional şi local', 'Buget', 26, 148, 142),
 (11266863, '121121021', 'CLUBUL COPIILOR CÂMPIA TURZII', 'CLUJ', 'CÂMPIA TURZII', 'Publică de interes naţional şi local', 'Buget', 0, 0, 0),
 (11266864, '121121007', 'CLUBUL COPIILOR DEJ', 'CLUJ', 'DEJ', 'Publică de interes naţional şi local', 'Buget', 0, 0, 0),
 (11266865, '121121033', 'CLUBUL COPIILOR GHERLA', 'CLUJ', 'GHERLA', 'Publică de interes naţional şi local', 'Buget', 0, 0, 0),
 (11266866, '123003546', 'CLUBUL COPIILOR HUEDIN', 'CLUJ', 'HUEDIN', 'Publică de interes naţional şi local', 'Buget', 0, 0, 0),
 (11266867, '121121019', 'CLUBUL COPIILOR TURDA', 'CLUJ', 'TURDA', 'Publică 

In [8]:
#verific coloanele
columns_school_details_cj_team2 = [row[0] for row in cursor.description]
columns_school_details_cj_team2

['idSchool',
 'siruesCode',
 'shortName',
 'county',
 'locality',
 'propertyForm',
 'fundingForm',
 'studyFormationsCount',
 'studentsCount',
 'personnelCount']

In [9]:
#import datele intr-un DataFrame
result_dict_school_details_cj_team2 = {header: values for header, values in zip(columns_school_details_cj_team2, zip(*school_details_cj_team2))}
df_school_details_cj_team2 = pd.DataFrame(result_dict_school_details_cj_team2)
df_school_details_cj_team2

,idSchool,siruesCode,shortName,county,locality,propertyForm,fundingForm,studyFormationsCount,studentsCount,personnelCount
0,11266861,123007621,CASA CORPULUI DIDACTIC CLUJ,CLUJ,CLUJ-NAPOCA,Publică de interes naţional şi local,Buget,0,0,100
1,11266905,12000012,C.J.R.A.E. CLUJ,CLUJ,CLUJ-NAPOCA,Publică de interes naţional şi local,Buget,0,0,157
2,11266906,120447612,CSEI CJ (SC. SPEC. 1),CLUJ,CLUJ-NAPOCA,Publică de interes naţional şi local,Buget,26,148,142
3,11266863,121121021,CLUBUL COPIILOR CÂMPIA TURZII,CLUJ,CÂMPIA TURZII,Publică de interes naţional şi local,Buget,0,0,0
4,11266864,121121007,CLUBUL COPIILOR DEJ,CLUJ,DEJ,Publică de interes naţional şi local,Buget,0,0,0
...,...,...,...,...,...,...,...,...,...,...
439,11267358,None,ȘCOALA PRIMARĂ WALDORF TURDA,CLUJ,TURDA,Publică de interes naţional şi local,Buget,8,0,0
440,11267360,120440717,ȘCOALA PROFESIONALĂ POIANA TURDA,CLUJ,TURDA,Publică de interes naţional şi local,Buget,26,411,39
441,11267345,120438829,"ȘCOALA PROFESIONALĂ RĂSCRUCI, COM. BONȚIDA",CLUJ,RĂSCRUCI,Publică de interes naţional şi local,Buget,23,250,42
442,11267124,120849509,"ȘCOALA PROFESIONALĂ SPECIALĂ ""SAMUS"" CLUJ - NA...",CLUJ,CLUJ-NAPOCA,Publică de interes naţional şi local,Buget,37,245,123


In [10]:
#sterg coloana deoarece contine o sigura valoare
df_school_details_cj_team2 = df_school_details_cj_team2.drop('propertyForm', axis=1) 
df_school_details_cj_team2

In [12]:
#sterg coloana deoarece contine o sigura valoare
df_school_details_cj_team2 = df_school_details_cj_team2.drop('fundingForm', axis=1)
df_school_details_cj_team2

,idSchool,siruesCode,shortName,county,locality,studyFormationsCount,studentsCount,personnelCount
0,11266861,123007621,CASA CORPULUI DIDACTIC CLUJ,CLUJ,CLUJ-NAPOCA,0,0,100
1,11266905,12000012,C.J.R.A.E. CLUJ,CLUJ,CLUJ-NAPOCA,0,0,157
2,11266906,120447612,CSEI CJ (SC. SPEC. 1),CLUJ,CLUJ-NAPOCA,26,148,142
3,11266863,121121021,CLUBUL COPIILOR CÂMPIA TURZII,CLUJ,CÂMPIA TURZII,0,0,0
4,11266864,121121007,CLUBUL COPIILOR DEJ,CLUJ,DEJ,0,0,0
...,...,...,...,...,...,...,...,...
439,11267358,None,ȘCOALA PRIMARĂ WALDORF TURDA,CLUJ,TURDA,8,0,0
440,11267360,120440717,ȘCOALA PROFESIONALĂ POIANA TURDA,CLUJ,TURDA,26,411,39
441,11267345,120438829,"ȘCOALA PROFESIONALĂ RĂSCRUCI, COM. BONȚIDA",CLUJ,RĂSCRUCI,23,250,42
442,11267124,120849509,"ȘCOALA PROFESIONALĂ SPECIALĂ ""SAMUS"" CLUJ - NA...",CLUJ,CLUJ-NAPOCA,37,245,123


In [13]:
#inlocuiesc diacriticele
carac_shortname = {"Ă":"A","Ș":"S","Ț":"T","Â":"A","Î":"I"}

#creez un tabel de traducere
translation_table1 = str.maketrans(carac_shortname)

#aplic la nivel de coloana
df_school_details_cj_team2["shortName"] = df_school_details_cj_team2["shortName"].str.translate(translation_table1)
df_school_details_cj_team2

,idSchool,siruesCode,shortName,county,locality,studyFormationsCount,studentsCount,personnelCount
0,11266861,123007621,CASA CORPULUI DIDACTIC CLUJ,CLUJ,CLUJ-NAPOCA,0,0,100
1,11266905,12000012,C.J.R.A.E. CLUJ,CLUJ,CLUJ-NAPOCA,0,0,157
2,11266906,120447612,CSEI CJ (SC. SPEC. 1),CLUJ,CLUJ-NAPOCA,26,148,142
3,11266863,121121021,CLUBUL COPIILOR CAMPIA TURZII,CLUJ,CÂMPIA TURZII,0,0,0
4,11266864,121121007,CLUBUL COPIILOR DEJ,CLUJ,DEJ,0,0,0
...,...,...,...,...,...,...,...,...
439,11267358,None,SCOALA PRIMARA WALDORF TURDA,CLUJ,TURDA,8,0,0
440,11267360,120440717,SCOALA PROFESIONALA POIANA TURDA,CLUJ,TURDA,26,411,39
441,11267345,120438829,"SCOALA PROFESIONALA RASCRUCI, COM. BONTIDA",CLUJ,RĂSCRUCI,23,250,42
442,11267124,120849509,"SCOALA PROFESIONALA SPECIALA ""SAMUS"" CLUJ - NA...",CLUJ,CLUJ-NAPOCA,37,245,123


In [14]:
#inlocuiesc diacriticele
carac_locality = {"Ă":"A","Ș":"S","Ț":"T","Â":"A","Î":"I"}

#creez un tabel de traducere
translation_table2 = str.maketrans(carac_locality)

#aplic la nivel de coloana
df_school_details_cj_team2["locality"] = df_school_details_cj_team2["locality"].str.translate(translation_table2)
df_school_details_cj_team2

,idSchool,siruesCode,shortName,county,locality,studyFormationsCount,studentsCount,personnelCount
0,11266861,123007621,CASA CORPULUI DIDACTIC CLUJ,CLUJ,CLUJ-NAPOCA,0,0,100
1,11266905,12000012,C.J.R.A.E. CLUJ,CLUJ,CLUJ-NAPOCA,0,0,157
2,11266906,120447612,CSEI CJ (SC. SPEC. 1),CLUJ,CLUJ-NAPOCA,26,148,142
3,11266863,121121021,CLUBUL COPIILOR CAMPIA TURZII,CLUJ,CAMPIA TURZII,0,0,0
4,11266864,121121007,CLUBUL COPIILOR DEJ,CLUJ,DEJ,0,0,0
...,...,...,...,...,...,...,...,...
439,11267358,None,SCOALA PRIMARA WALDORF TURDA,CLUJ,TURDA,8,0,0
440,11267360,120440717,SCOALA PROFESIONALA POIANA TURDA,CLUJ,TURDA,26,411,39
441,11267345,120438829,"SCOALA PROFESIONALA RASCRUCI, COM. BONTIDA",CLUJ,RASCRUCI,23,250,42
442,11267124,120849509,"SCOALA PROFESIONALA SPECIALA ""SAMUS"" CLUJ - NA...",CLUJ,CLUJ-NAPOCA,37,245,123


In [15]:
#verific tipul coloanelor ca acestea sa indeplineasca cerintele pentru analiza
column_dtypes = df_school_details_cj_team2.dtypes
column_dtypes

idSchool                 int64
siruesCode              object
shortName               object
county                  object
locality                object
studyFormationsCount     int64
studentsCount            int64
personnelCount           int64
dtype: object

In [16]:
#convertesc in format int
df_school_details_cj_team2['siruesCode'] = 
pd.to_numeric(df_school_details_cj_team2['siruesCode'], errors='coerce').astype('Int64')

In [17]:
#verific tipul coloanelor dupa conversie
column_dtypes = df_school_details_cj_team2.dtypes 
column_dtypes

idSchool                 int64
siruesCode               Int64
shortName               object
county                  object
locality                object
studyFormationsCount     int64
studentsCount            int64
personnelCount           int64
dtype: object

### Preluarea tabelului cu informatii despre starea cladirilor scolare din Romania

In [18]:
#creez un query pentru import
query = 'SELECT * FROM school_buildings_team2'
cursor.execute(query)
school_buildings_team2 = cursor.fetchall()
school_buildings_team2

[(13902, 11263103, 'CASA CORPULUI DIDACTIC MEHEDINTI', 'Parţial reabilitată', 'Cărămidă', 'Ţiglă metalică', 'Reabilitată', 'Nereabilitată', 'Lemn', 'Lemn', -2208988800000),
 (2083, 11269004, 'Clădirea A', 'Nereabilitată', 'Cărămidă', 'Ţiglă ceramică', 'Nereabilitată', 'Nereabilitată', 'Lemn', 'Lemn', -2208988800000),
 (4390, 11271116, 'CASA CORPULUI DIDACTIC', 'Reabilitată', 'Cărămidă', 'Tablă', 'Nu necesită reabilitare', 'Reabilitată', 'Aluminiu', 'PVC', -2208988800000),
 (1595, 11260805, 'CLADIRE A', 'Reabilitată', 'Cărămidă', 'Tablă', 'Reabilitată', 'Reabilitată', 'PVC', 'Lemn', -2208988800000),
 (18084, 11267362, 'INSPECTORATUL SCOLAR JUDETEAN VALCEA', 'Reabilitată', 'Cadru beton', 'Ţiglă beton', 'Reabilitată', 'Reabilitată', 'PVC', 'Lemn', -2208988800000),
 (21840, 11263948, 'CASA FAMILIALA ANDREI', 'Nu necesită reabilitare', 'Cărămidă', 'Ţiglă ceramică', 'Nu necesită reabilitare', 'Nu necesită reabilitare', 'PVC', 'Lemn', -2208988800000),
 (16790, 11263948, 'PAVILIONUL B8', 'Nere

In [19]:
#verific coloanele pentru a decide pe care le pastrez
columns_school_buildings_team2 = [row[0] for row in cursor.description]
columns_school_buildings_team2

['id',
 'idSchool',
 'buildingDescription',
 'generalState',
 'buildingMaterial',
 'roofMaterial',
 'roofCondition',
 'joineryCondition',
 'windowJoinery',
 'doorJoinery',
 'dateFrom']

In [20]:
#import datele intr-un DataFrame
result_dict_school_buildings_team2 = {header: values for header, values in zip(columns_school_buildings_team2, zip(*school_buildings_team2))}
df_school_buildings_team2 = pd.DataFrame(result_dict_school_buildings_team2)
df_school_buildings_team2

,id,idSchool,buildingDescription,generalState,buildingMaterial,roofMaterial,roofCondition,joineryCondition,windowJoinery,doorJoinery,dateFrom
0,13902,11263103,CASA CORPULUI DIDACTIC MEHEDINTI,Parţial reabilitată,Cărămidă,Ţiglă metalică,Reabilitată,Nereabilitată,Lemn,Lemn,-2208988800000
1,2083,11269004,Clădirea A,Nereabilitată,Cărămidă,Ţiglă ceramică,Nereabilitată,Nereabilitată,Lemn,Lemn,-2208988800000
2,4390,11271116,CASA CORPULUI DIDACTIC,Reabilitată,Cărămidă,Tablă,Nu necesită reabilitare,Reabilitată,Aluminiu,PVC,-2208988800000
3,1595,11260805,CLADIRE A,Reabilitată,Cărămidă,Tablă,Reabilitată,Reabilitată,PVC,Lemn,-2208988800000
4,18084,11267362,INSPECTORATUL SCOLAR JUDETEAN VALCEA,Reabilitată,Cadru beton,Ţiglă beton,Reabilitată,Reabilitată,PVC,Lemn,-2208988800000
...,...,...,...,...,...,...,...,...,...,...,...
14953,17888,11261173,Sala Sport,Nereabilitată,Cărămidă,Tablă,Alte situaţii,Reabilitată,PVC,PVC,-2208988800000
14954,22346,11261173,Atelier Mecanic,Nereabilitată,Cărămidă,Alte materiale,Nereabilitată,Reabilitată,PVC,Lemn,-2208988800000
14955,22338,11261173,Cabina Poarta,Nereabilitată,Cărămidă,Tablă,Nereabilitată,Nereabilitată,Lemn,Lemn,-2208988800000
14956,22334,11261173,Cabina Poarta,Nereabilitată,Cărămidă,Tablă,Nereabilitată,Reabilitată,PVC,PVC,-2208988800000


In [21]:
#sterg coloana deoarece nu este relevanta pentru analiza
df_school_buildings_team2 = df_school_buildings_team2.drop('id', axis=1)
df_school_buildings_team2

,idSchool,buildingDescription,generalState,buildingMaterial,roofMaterial,roofCondition,joineryCondition,windowJoinery,doorJoinery,dateFrom
0,11263103,CASA CORPULUI DIDACTIC MEHEDINTI,Parţial reabilitată,Cărămidă,Ţiglă metalică,Reabilitată,Nereabilitată,Lemn,Lemn,-2208988800000
1,11269004,Clădirea A,Nereabilitată,Cărămidă,Ţiglă ceramică,Nereabilitată,Nereabilitată,Lemn,Lemn,-2208988800000
2,11271116,CASA CORPULUI DIDACTIC,Reabilitată,Cărămidă,Tablă,Nu necesită reabilitare,Reabilitată,Aluminiu,PVC,-2208988800000
3,11260805,CLADIRE A,Reabilitată,Cărămidă,Tablă,Reabilitată,Reabilitată,PVC,Lemn,-2208988800000
4,11267362,INSPECTORATUL SCOLAR JUDETEAN VALCEA,Reabilitată,Cadru beton,Ţiglă beton,Reabilitată,Reabilitată,PVC,Lemn,-2208988800000
...,...,...,...,...,...,...,...,...,...,...
14953,11261173,Sala Sport,Nereabilitată,Cărămidă,Tablă,Alte situaţii,Reabilitată,PVC,PVC,-2208988800000
14954,11261173,Atelier Mecanic,Nereabilitată,Cărămidă,Alte materiale,Nereabilitată,Reabilitată,PVC,Lemn,-2208988800000
14955,11261173,Cabina Poarta,Nereabilitată,Cărămidă,Tablă,Nereabilitată,Nereabilitată,Lemn,Lemn,-2208988800000
14956,11261173,Cabina Poarta,Nereabilitată,Cărămidă,Tablă,Nereabilitată,Reabilitată,PVC,PVC,-2208988800000


In [22]:
#sterg coloana deoarece nu este relevanta pentru analiza
df_school_buildings_team2 = df_school_buildings_team2.drop('dateFrom', axis=1)
df_school_buildings_team2

,idSchool,buildingDescription,generalState,buildingMaterial,roofMaterial,roofCondition,joineryCondition,windowJoinery,doorJoinery
0,11263103,CASA CORPULUI DIDACTIC MEHEDINTI,Parţial reabilitată,Cărămidă,Ţiglă metalică,Reabilitată,Nereabilitată,Lemn,Lemn
1,11269004,Clădirea A,Nereabilitată,Cărămidă,Ţiglă ceramică,Nereabilitată,Nereabilitată,Lemn,Lemn
2,11271116,CASA CORPULUI DIDACTIC,Reabilitată,Cărămidă,Tablă,Nu necesită reabilitare,Reabilitată,Aluminiu,PVC
3,11260805,CLADIRE A,Reabilitată,Cărămidă,Tablă,Reabilitată,Reabilitată,PVC,Lemn
4,11267362,INSPECTORATUL SCOLAR JUDETEAN VALCEA,Reabilitată,Cadru beton,Ţiglă beton,Reabilitată,Reabilitată,PVC,Lemn
...,...,...,...,...,...,...,...,...,...
14953,11261173,Sala Sport,Nereabilitată,Cărămidă,Tablă,Alte situaţii,Reabilitată,PVC,PVC
14954,11261173,Atelier Mecanic,Nereabilitată,Cărămidă,Alte materiale,Nereabilitată,Reabilitată,PVC,Lemn
14955,11261173,Cabina Poarta,Nereabilitată,Cărămidă,Tablă,Nereabilitată,Nereabilitată,Lemn,Lemn
14956,11261173,Cabina Poarta,Nereabilitată,Cărămidă,Tablă,Nereabilitată,Reabilitată,PVC,PVC


In [23]:
#transform in majuscule pentru a avea cursivitate
df_school_buildings_team2['buildingDescription'] = df_school_buildings_team2['buildingDescription'].str.upper()
df_school_buildings_team2

,idSchool,buildingDescription,generalState,buildingMaterial,roofMaterial,roofCondition,joineryCondition,windowJoinery,doorJoinery
0,11263103,CASA CORPULUI DIDACTIC MEHEDINTI,Parţial reabilitată,Cărămidă,Ţiglă metalică,Reabilitată,Nereabilitată,Lemn,Lemn
1,11269004,CLĂDIREA A,Nereabilitată,Cărămidă,Ţiglă ceramică,Nereabilitată,Nereabilitată,Lemn,Lemn
2,11271116,CASA CORPULUI DIDACTIC,Reabilitată,Cărămidă,Tablă,Nu necesită reabilitare,Reabilitată,Aluminiu,PVC
3,11260805,CLADIRE A,Reabilitată,Cărămidă,Tablă,Reabilitată,Reabilitată,PVC,Lemn
4,11267362,INSPECTORATUL SCOLAR JUDETEAN VALCEA,Reabilitată,Cadru beton,Ţiglă beton,Reabilitată,Reabilitată,PVC,Lemn
...,...,...,...,...,...,...,...,...,...
14953,11261173,SALA SPORT,Nereabilitată,Cărămidă,Tablă,Alte situaţii,Reabilitată,PVC,PVC
14954,11261173,ATELIER MECANIC,Nereabilitată,Cărămidă,Alte materiale,Nereabilitată,Reabilitată,PVC,Lemn
14955,11261173,CABINA POARTA,Nereabilitată,Cărămidă,Tablă,Nereabilitată,Nereabilitată,Lemn,Lemn
14956,11261173,CABINA POARTA,Nereabilitată,Cărămidă,Tablă,Nereabilitată,Reabilitată,PVC,PVC


In [24]:
#inlocuiesc diacriticele
carac_buildingDescription = {"Ă":"A","Ș":"S","Ț":"T","Â":"A","Î":"I"}

#creez un tabel de traducere
translation_table3 = str.maketrans(carac_buildingDescription)

#aplic la nivel de coloana
df_school_buildings_team2["buildingDescription"] = df_school_buildings_team2["buildingDescription"].str.translate(translation_table3)
df_school_buildings_team2

,idSchool,buildingDescription,generalState,buildingMaterial,roofMaterial,roofCondition,joineryCondition,windowJoinery,doorJoinery
0,11263103,CASA CORPULUI DIDACTIC MEHEDINTI,Parţial reabilitată,Cărămidă,Ţiglă metalică,Reabilitată,Nereabilitată,Lemn,Lemn
1,11269004,CLADIREA A,Nereabilitată,Cărămidă,Ţiglă ceramică,Nereabilitată,Nereabilitată,Lemn,Lemn
2,11271116,CASA CORPULUI DIDACTIC,Reabilitată,Cărămidă,Tablă,Nu necesită reabilitare,Reabilitată,Aluminiu,PVC
3,11260805,CLADIRE A,Reabilitată,Cărămidă,Tablă,Reabilitată,Reabilitată,PVC,Lemn
4,11267362,INSPECTORATUL SCOLAR JUDETEAN VALCEA,Reabilitată,Cadru beton,Ţiglă beton,Reabilitată,Reabilitată,PVC,Lemn
...,...,...,...,...,...,...,...,...,...
14953,11261173,SALA SPORT,Nereabilitată,Cărămidă,Tablă,Alte situaţii,Reabilitată,PVC,PVC
14954,11261173,ATELIER MECANIC,Nereabilitată,Cărămidă,Alte materiale,Nereabilitată,Reabilitată,PVC,Lemn
14955,11261173,CABINA POARTA,Nereabilitată,Cărămidă,Tablă,Nereabilitată,Nereabilitată,Lemn,Lemn
14956,11261173,CABINA POARTA,Nereabilitată,Cărămidă,Tablă,Nereabilitată,Reabilitată,PVC,PVC


In [25]:
#generez valorile unice din coloana
unique_generalState = df_school_buildings_team2['generalState'].unique()
unique_generalState

array(['Parţial reabilitată', 'Nereabilitată', 'Reabilitată',
       'Nu necesită reabilitare', 'În curs de reabilitare'], dtype=object)

In [ ]:
## Voi converti string-urile urmatoare in valori numerice pentru a le putea utiliza in analiza exploratorie
## le-am numerotat dupa logica pozitiv = 1
## Reabilitata = 1
## Parțial reabilitata = 2
## În curs de reabilitare = 3
## Nu necesita reabilitare = 4
## Nereabilitata = 5
## Alte situații = 6

In [26]:
#construiesc dictionarul si inlocuiesc cu caractere numerice
replacement_mapping = {'Reabilitată':1,'Parţial reabilitată':2,'În curs de reabilitare':3,'Nu necesită reabilitare':4,'Nereabilitată':5}
df_school_buildings_team2['generalState'] = df_school_buildings_team2['generalState'].replace(replacement_mapping)
df_school_buildings_team2

,idSchool,buildingDescription,generalState,buildingMaterial,roofMaterial,roofCondition,joineryCondition,windowJoinery,doorJoinery
0,11263103,CASA CORPULUI DIDACTIC MEHEDINTI,2,Cărămidă,Ţiglă metalică,Reabilitată,Nereabilitată,Lemn,Lemn
1,11269004,CLADIREA A,5,Cărămidă,Ţiglă ceramică,Nereabilitată,Nereabilitată,Lemn,Lemn
2,11271116,CASA CORPULUI DIDACTIC,1,Cărămidă,Tablă,Nu necesită reabilitare,Reabilitată,Aluminiu,PVC
3,11260805,CLADIRE A,1,Cărămidă,Tablă,Reabilitată,Reabilitată,PVC,Lemn
4,11267362,INSPECTORATUL SCOLAR JUDETEAN VALCEA,1,Cadru beton,Ţiglă beton,Reabilitată,Reabilitată,PVC,Lemn
...,...,...,...,...,...,...,...,...,...
14953,11261173,SALA SPORT,5,Cărămidă,Tablă,Alte situaţii,Reabilitată,PVC,PVC
14954,11261173,ATELIER MECANIC,5,Cărămidă,Alte materiale,Nereabilitată,Reabilitată,PVC,Lemn
14955,11261173,CABINA POARTA,5,Cărămidă,Tablă,Nereabilitată,Nereabilitată,Lemn,Lemn
14956,11261173,CABINA POARTA,5,Cărămidă,Tablă,Nereabilitată,Reabilitată,PVC,PVC


In [27]:
#generez valorile unice din coloana
unique_roofCondition = df_school_buildings_team2['roofCondition'].unique()
unique_roofCondition

array(['Reabilitată', 'Nereabilitată', 'Nu necesită reabilitare',
       'Parţial reabilitată', 'În curs de reabilitare', 'Alte situaţii'],
      dtype=object)

In [28]:
#construiesc dictionarul si inlocuiesc cu caractere numerice
replacement_mapping_ext = {'Reabilitată':1,'Parţial reabilitată':2,'În curs de reabilitare':3,'Nu necesită reabilitare':4,'Nereabilitată':5, 'Alte situaţii':6}
df_school_buildings_team2['roofCondition'] = df_school_buildings_team2['roofCondition'].replace(replacement_mapping_ext)
df_school_buildings_team2

,idSchool,buildingDescription,generalState,buildingMaterial,roofMaterial,roofCondition,joineryCondition,windowJoinery,doorJoinery
0,11263103,CASA CORPULUI DIDACTIC MEHEDINTI,2,Cărămidă,Ţiglă metalică,1,Nereabilitată,Lemn,Lemn
1,11269004,CLADIREA A,5,Cărămidă,Ţiglă ceramică,5,Nereabilitată,Lemn,Lemn
2,11271116,CASA CORPULUI DIDACTIC,1,Cărămidă,Tablă,4,Reabilitată,Aluminiu,PVC
3,11260805,CLADIRE A,1,Cărămidă,Tablă,1,Reabilitată,PVC,Lemn
4,11267362,INSPECTORATUL SCOLAR JUDETEAN VALCEA,1,Cadru beton,Ţiglă beton,1,Reabilitată,PVC,Lemn
...,...,...,...,...,...,...,...,...,...
14953,11261173,SALA SPORT,5,Cărămidă,Tablă,6,Reabilitată,PVC,PVC
14954,11261173,ATELIER MECANIC,5,Cărămidă,Alte materiale,5,Reabilitată,PVC,Lemn
14955,11261173,CABINA POARTA,5,Cărămidă,Tablă,5,Nereabilitată,Lemn,Lemn
14956,11261173,CABINA POARTA,5,Cărămidă,Tablă,5,Reabilitată,PVC,PVC


In [29]:
#generez valorile unice din coloana
unique_joineryCondition = df_school_buildings_team2['joineryCondition'].unique()
unique_joineryCondition

array(['Nereabilitată', 'Reabilitată', 'Nu necesită reabilitare',
       'Parţial reabilitată', 'Alte situaţii', 'În curs de reabilitare'],
      dtype=object)

In [30]:
#inlocuiesc cu caractere numerice
df_school_buildings_team2['joineryCondition'] = df_school_buildings_team2['joineryCondition'].replace(replacement_mapping_ext) #inlocuiesc cu caractere numerice
df_school_buildings_team2

,idSchool,buildingDescription,generalState,buildingMaterial,roofMaterial,roofCondition,joineryCondition,windowJoinery,doorJoinery
0,11263103,CASA CORPULUI DIDACTIC MEHEDINTI,2,Cărămidă,Ţiglă metalică,1,5,Lemn,Lemn
1,11269004,CLADIREA A,5,Cărămidă,Ţiglă ceramică,5,5,Lemn,Lemn
2,11271116,CASA CORPULUI DIDACTIC,1,Cărămidă,Tablă,4,1,Aluminiu,PVC
3,11260805,CLADIRE A,1,Cărămidă,Tablă,1,1,PVC,Lemn
4,11267362,INSPECTORATUL SCOLAR JUDETEAN VALCEA,1,Cadru beton,Ţiglă beton,1,1,PVC,Lemn
...,...,...,...,...,...,...,...,...,...
14953,11261173,SALA SPORT,5,Cărămidă,Tablă,6,1,PVC,PVC
14954,11261173,ATELIER MECANIC,5,Cărămidă,Alte materiale,5,1,PVC,Lemn
14955,11261173,CABINA POARTA,5,Cărămidă,Tablă,5,5,Lemn,Lemn
14956,11261173,CABINA POARTA,5,Cărămidă,Tablă,5,1,PVC,PVC


In [31]:
#transform in majuscule
df_school_buildings_team2['buildingMaterial'] = df_school_buildings_team2['buildingMaterial'].str.upper()

#inlocuiesc diacriticele
carac_buildingMaterial = {"Ă":"A","Ș":"S","Ț":"T","Â":"A","Î":"I"}

#creez un tabel de traducere
translation_table4 = str.maketrans(carac_buildingMaterial)

#aplic la nivel de coloana
df_school_buildings_team2["buildingMaterial"] = df_school_buildings_team2["buildingMaterial"].str.translate(translation_table4)
df_school_buildings_team2

,idSchool,buildingDescription,generalState,buildingMaterial,roofMaterial,roofCondition,joineryCondition,windowJoinery,doorJoinery
0,11263103,CASA CORPULUI DIDACTIC MEHEDINTI,2,CARAMIDA,Ţiglă metalică,1,5,Lemn,Lemn
1,11269004,CLADIREA A,5,CARAMIDA,Ţiglă ceramică,5,5,Lemn,Lemn
2,11271116,CASA CORPULUI DIDACTIC,1,CARAMIDA,Tablă,4,1,Aluminiu,PVC
3,11260805,CLADIRE A,1,CARAMIDA,Tablă,1,1,PVC,Lemn
4,11267362,INSPECTORATUL SCOLAR JUDETEAN VALCEA,1,CADRU BETON,Ţiglă beton,1,1,PVC,Lemn
...,...,...,...,...,...,...,...,...,...
14953,11261173,SALA SPORT,5,CARAMIDA,Tablă,6,1,PVC,PVC
14954,11261173,ATELIER MECANIC,5,CARAMIDA,Alte materiale,5,1,PVC,Lemn
14955,11261173,CABINA POARTA,5,CARAMIDA,Tablă,5,5,Lemn,Lemn
14956,11261173,CABINA POARTA,5,CARAMIDA,Tablă,5,1,PVC,PVC


In [32]:
#transform in majuscule
df_school_buildings_team2['roofMaterial'] = df_school_buildings_team2['roofMaterial'].str.upper()

#inlocuiesc diacriticele
carac_roofMaterial = {"Ă":"A","Ș":"S","Ţ":"T","Â":"A","Î":"I"}

#creez un tabel de traducere
translation_table5 = str.maketrans(carac_roofMaterial)

#aplic la nivel de coloana
df_school_buildings_team2["roofMaterial"] = df_school_buildings_team2["roofMaterial"].str.translate(translation_table5)
df_school_buildings_team2

,idSchool,buildingDescription,generalState,buildingMaterial,roofMaterial,roofCondition,joineryCondition,windowJoinery,doorJoinery
0,11263103,CASA CORPULUI DIDACTIC MEHEDINTI,2,CARAMIDA,TIGLA METALICA,1,5,Lemn,Lemn
1,11269004,CLADIREA A,5,CARAMIDA,TIGLA CERAMICA,5,5,Lemn,Lemn
2,11271116,CASA CORPULUI DIDACTIC,1,CARAMIDA,TABLA,4,1,Aluminiu,PVC
3,11260805,CLADIRE A,1,CARAMIDA,TABLA,1,1,PVC,Lemn
4,11267362,INSPECTORATUL SCOLAR JUDETEAN VALCEA,1,CADRU BETON,TIGLA BETON,1,1,PVC,Lemn
...,...,...,...,...,...,...,...,...,...
14953,11261173,SALA SPORT,5,CARAMIDA,TABLA,6,1,PVC,PVC
14954,11261173,ATELIER MECANIC,5,CARAMIDA,ALTE MATERIALE,5,1,PVC,Lemn
14955,11261173,CABINA POARTA,5,CARAMIDA,TABLA,5,5,Lemn,Lemn
14956,11261173,CABINA POARTA,5,CARAMIDA,TABLA,5,1,PVC,PVC


In [33]:
#transform in majuscule
df_school_buildings_team2['windowJoinery'] = df_school_buildings_team2['windowJoinery'].str.upper()

#inlocuiesc diacriticele
carac_windowJoinery = {"Ă":"A","Ș":"S","Ţ":"T","Â":"A","Î":"I"}

#creez un tabel de traducere
translation_table6 = str.maketrans(carac_windowJoinery)

#aplic la nivel de coloana
df_school_buildings_team2["windowJoinery"] = df_school_buildings_team2["windowJoinery"].str.translate(translation_table6)
df_school_buildings_team2

,idSchool,buildingDescription,generalState,buildingMaterial,roofMaterial,roofCondition,joineryCondition,windowJoinery,doorJoinery
0,11263103,CASA CORPULUI DIDACTIC MEHEDINTI,2,CARAMIDA,TIGLA METALICA,1,5,LEMN,Lemn
1,11269004,CLADIREA A,5,CARAMIDA,TIGLA CERAMICA,5,5,LEMN,Lemn
2,11271116,CASA CORPULUI DIDACTIC,1,CARAMIDA,TABLA,4,1,ALUMINIU,PVC
3,11260805,CLADIRE A,1,CARAMIDA,TABLA,1,1,PVC,Lemn
4,11267362,INSPECTORATUL SCOLAR JUDETEAN VALCEA,1,CADRU BETON,TIGLA BETON,1,1,PVC,Lemn
...,...,...,...,...,...,...,...,...,...
14953,11261173,SALA SPORT,5,CARAMIDA,TABLA,6,1,PVC,PVC
14954,11261173,ATELIER MECANIC,5,CARAMIDA,ALTE MATERIALE,5,1,PVC,Lemn
14955,11261173,CABINA POARTA,5,CARAMIDA,TABLA,5,5,LEMN,Lemn
14956,11261173,CABINA POARTA,5,CARAMIDA,TABLA,5,1,PVC,PVC


In [34]:
#transform in majuscule
df_school_buildings_team2['doorJoinery'] = df_school_buildings_team2['doorJoinery'].str.upper() 

#inlocuiesc diacriticele
carac_doorJoinery = {"Ă":"A","Ș":"S","Ţ":"T","Â":"A","Î":"I"}

#creez un tabel de traducere
translation_table7 = str.maketrans(carac_doorJoinery)

#aplic la nivel de coloana
df_school_buildings_team2["doorJoinery"] = df_school_buildings_team2["doorJoinery"].str.translate(translation_table7)
df_school_buildings_team2

,idSchool,buildingDescription,generalState,buildingMaterial,roofMaterial,roofCondition,joineryCondition,windowJoinery,doorJoinery
0,11263103,CASA CORPULUI DIDACTIC MEHEDINTI,2,CARAMIDA,TIGLA METALICA,1,5,LEMN,LEMN
1,11269004,CLADIREA A,5,CARAMIDA,TIGLA CERAMICA,5,5,LEMN,LEMN
2,11271116,CASA CORPULUI DIDACTIC,1,CARAMIDA,TABLA,4,1,ALUMINIU,PVC
3,11260805,CLADIRE A,1,CARAMIDA,TABLA,1,1,PVC,LEMN
4,11267362,INSPECTORATUL SCOLAR JUDETEAN VALCEA,1,CADRU BETON,TIGLA BETON,1,1,PVC,LEMN
...,...,...,...,...,...,...,...,...,...
14953,11261173,SALA SPORT,5,CARAMIDA,TABLA,6,1,PVC,PVC
14954,11261173,ATELIER MECANIC,5,CARAMIDA,ALTE MATERIALE,5,1,PVC,LEMN
14955,11261173,CABINA POARTA,5,CARAMIDA,TABLA,5,5,LEMN,LEMN
14956,11261173,CABINA POARTA,5,CARAMIDA,TABLA,5,1,PVC,PVC


In [35]:
#verific tipul coloanelor
column_dtypes = df_school_buildings_team2.dtypes
column_dtypes

idSchool                int64
buildingDescription    object
generalState            int64
buildingMaterial       object
roofMaterial           object
roofCondition           int64
joineryCondition        int64
windowJoinery          object
doorJoinery            object
dtype: object

### Preluarea tabelului cu informatii despre codul SIIIR pentru a putea uni tabelele cu rezultatele de la evaluarea nationala

In [36]:
#creez un query pentru import
query = 'SELECT * FROM sirues_siiir_team2'
cursor.execute(query)
sirues_siiir_team2 = cursor.fetchall()
sirues_siiir_team2

[(121121007, 'CLUBUL COPIILOR DEJ', 1252103536),
 (121266467, 'CLUBUL Č\x98COLAR SPORTIV "VIITORUL" CLUJ-NAPOCA', 1281102324),
 (121181148, 'COLEGIUL ECONOMIC "IULIAN POP" CLUJ-NAPOCA', 1261103197),
 (120844602, 'COLEGIUL "EMIL NEGRUČšIU" TURDA', 1261102341),
 (120437849, 'COLEGIUL NAČšIONAL "EMIL RACOVIČšÄ‚" CLUJ-NAPOCA', 1261105023),
 (120224125, 'COLEGIUL NAČšIONAL "GEORGE BARIČšIU" CLUJ-NAPOCA', 1261102567),
 (120437825, 'COLEGIUL NAČšIONAL "GEORGE COČ\x98BUC" CLUJ-NAPOCA', 1261103939),
 (120437813, 'COLEGIUL NAČšIONAL "GHEORGHE Č\x98INCAI" CLUJ-NAPOCA', 1261103473),
 (120437980, 'COLEGIUL NAČšIONAL "MIHAI VITEAZUL" TURDA', 1261100375),
 (120438087, 'COLEGIUL NAČšIONAL PEDAGOGIC "GHEORGHE LAZÄ‚R" CLUJ', 1261102703),
 (120438099, 'COLEGIUL TEHNIC DE CONSTRUCTII "ANGHEL SALIGNY"', 1261103274),
 (120224113, 'COLEGIUL TEHNIC "AUGUSTIN MAIOR" CLUJ', 1261103975),
 (120099312, 'COLEGIUL TEHNIC DE TRANSPORTURI "TRANSILVANIA"', 1261103256),
 (121181071, 'COLEGIUL DE SERVICII IN TURISM "NAPO

In [37]:
#verific coloanele
columns_sirues_siiir_team2 = [row[0] for row in cursor.description]
columns_sirues_siiir_team2

['siruesCode', 'Denumire', 'cod_SIIIR']

In [38]:
#import datele intr-un DataFrame
result_dict_sirues_siiir_team2 = {header: values for header, values in zip(columns_sirues_siiir_team2, zip(*sirues_siiir_team2))}
df_sirues_siiir_team2 = pd.DataFrame(result_dict_sirues_siiir_team2)
df_sirues_siiir_team2

,siruesCode,Denumire,cod_SIIIR
0,121121007,CLUBUL COPIILOR DEJ,1252103536
1,121266467,"CLUBUL ČCOLAR SPORTIV ""VIITORUL"" CLUJ-NAPOCA",1281102324
2,121181148,"COLEGIUL ECONOMIC ""IULIAN POP"" CLUJ-NAPOCA",1261103197
3,120844602,"COLEGIUL ""EMIL NEGRUČšIU"" TURDA",1261102341
4,120437849,"COLEGIUL NAČšIONAL ""EMIL RACOVIČšÄ‚"" CLUJ-NAPOCA",1261105023
...,...,...,...
147,120441242,"ČCOALA PRIMARÄ‚ VÄ‚LENI, COM. CÄ‚LÄ‚ČšELE",1262100651
148,120442076,"ČCOALA PRIMARÄ‚ VIČTEA, COM. GĂ‚RBÄ‚U",1262101634
149,120440717,ČCOALA PROFESIONALÄ‚ POIANA TURDA,1261103817
150,120438829,"ČCOALA PROFESIONALÄ‚ RÄ‚SCRUCI, COM. BONČšIDA",1261102377


In [39]:
#sterg coloana pentru ca denumirea scolilor exista deja in coloana 'shortname'
df_sirues_siiir_team2 = df_sirues_siiir_team2.drop('Denumire', axis=1)
df_sirues_siiir_team2

,siruesCode,cod_SIIIR
0,121121007,1252103536
1,121266467,1281102324
2,121181148,1261103197
3,120844602,1261102341
4,120437849,1261105023
...,...,...
147,120441242,1262100651
148,120442076,1262101634
149,120440717,1261103817
150,120438829,1261102377


In [40]:
#verific tipul coloanelor
column_dtypes = df_sirues_siiir_team2.dtypes
column_dtypes

siruesCode    int64
cod_SIIIR     int64
dtype: object

### Preluarea tabelului cu informatii despre rezultatele de la evaluarea nationala

In [41]:
#creez un query pentru import
query = 'SELECT * FROM rezultate_evaluare_nationala_2023_team2'
cursor.execute(query)
rezultate_evaluare_nationala_2023_team2 = cursor.fetchall()
rezultate_evaluare_nationala_2023_team2

[(2176401, 'M', 'RURAL', 1461100707, 'PREZENT', 'PREZENT', 'PREZENT', '6,85', '8', '8,25', 'NU', None, 'NU', None, 'NU', None, '6,85', '8', '8,25', '7,7', '9,22'),
 (2551851, 'M', 'RURAL', 1461100707, 'PREZENT', 'PREZENT', 'PREZENT', '2,75', '6,35', '6,45', 'NU', None, 'NU', None, 'NU', None, '2,75', '6,35', '6,45', '5,18', '8,12'),
 (2551551, 'M', 'RURAL', 1461100707, 'PREZENT', 'PREZENT', 'PREZENT', '1,8', '5,6', '3,55', 'NU', None, 'NU', None, 'NU', None, '1,8', '5,6', '3,55', '3,65', '8,52'),
 (2554651, 'F', 'RURAL', 1461100707, 'PREZENT', 'PREZENT', 'PREZENT', '4,35', '8,05', '6,95', 'NU', None, 'NU', None, 'NU', None, '4,35', '8,05', '6,95', '6,45', '9,25'),
 (681001, 'F', 'URBAN', 1461100716, 'PREZENT', 'PREZENT', 'PREZENT', '9,2', '9,35', '9,55', 'NU', None, 'NU', None, 'NU', None, '9,2', '9,35', '9,55', '9,36', '9,74'),
 (681601, 'F', 'URBAN', 1461100716, 'PREZENT', 'PREZENT', 'PREZENT', '5,5', '8,45', '5,9', 'NU', None, 'NU', None, 'NU', None, '5,5', '8,45', '5,9', '6,61', '8

In [42]:
#verific coloanele
columns_rezultate_evaluare_nationala_2023_team2 = [row[0] for row in cursor.description]
columns_rezultate_evaluare_nationala_2023_team2

['COD UNIC CANDIDAT',
 'SEX',
 'MEDIU',
 'cod_SIIIR',
 'STATUS ROMANA',
 ' STATUS LIMBA MATERNA',
 'STATUS MATEMATICA',
 'NOTA ROMANA',
 'NOTA LIMBA MATERNA',
 'NOTA MATEMATICA',
 'CONTESTATIE ROMANA',
 'NOTA CONTESTATIE ROMANA',
 'CONTESTATIE LIMBA MATERNA',
 'NOTA CONTESTATIE LB MATERNA',
 'CONTESTATIE MATEMATICA',
 'NOTA CONTESTATIE MATEMATICA',
 'NOTA FINALA ROMANA',
 'NOTA FINALA LB MATERNA',
 'NOTA FINALA MATEMATICA',
 'MEDIA',
 'MEDIA V-VIII']

In [43]:
#import datele intr-un DataFrame
result_dict_rezultate_evaluare_nationala_2023_team2 = {header: values for header, values in zip(columns_rezultate_evaluare_nationala_2023_team2, 
                                                                                                zip(*rezultate_evaluare_nationala_2023_team2))}
df_rezultate_evaluare_nationala_2023_team2 = pd.DataFrame(result_dict_rezultate_evaluare_nationala_2023_team2)
df_rezultate_evaluare_nationala_2023_team2.head(10)

,COD UNIC CANDIDAT,SEX,MEDIU,cod_SIIIR,STATUS ROMANA,STATUS LIMBA MATERNA,STATUS MATEMATICA,NOTA ROMANA,NOTA LIMBA MATERNA,NOTA MATEMATICA,...,NOTA CONTESTATIE ROMANA,CONTESTATIE LIMBA MATERNA,NOTA CONTESTATIE LB MATERNA,CONTESTATIE MATEMATICA,NOTA CONTESTATIE MATEMATICA,NOTA FINALA ROMANA,NOTA FINALA LB MATERNA,NOTA FINALA MATEMATICA,MEDIA,MEDIA V-VIII
0,2176401,M,RURAL,1461100707,PREZENT,PREZENT,PREZENT,"6,85",8,"8,25",...,None,NU,None,NU,None,"6,85",8,"8,25","7,7","9,22"
1,2551851,M,RURAL,1461100707,PREZENT,PREZENT,PREZENT,"2,75","6,35","6,45",...,None,NU,None,NU,None,"2,75","6,35","6,45","5,18","8,12"
2,2551551,M,RURAL,1461100707,PREZENT,PREZENT,PREZENT,"1,8","5,6","3,55",...,None,NU,None,NU,None,"1,8","5,6","3,55","3,65","8,52"
3,2554651,F,RURAL,1461100707,PREZENT,PREZENT,PREZENT,"4,35","8,05","6,95",...,None,NU,None,NU,None,"4,35","8,05","6,95","6,45","9,25"
4,681001,F,URBAN,1461100716,PREZENT,PREZENT,PREZENT,"9,2","9,35","9,55",...,None,NU,None,NU,None,"9,2","9,35","9,55","9,36","9,74"
5,681601,F,URBAN,1461100716,PREZENT,PREZENT,PREZENT,"5,5","8,45","5,9",...,None,NU,None,NU,None,"5,5","8,45","5,9","6,61","8,49"
6,681651,M,URBAN,1461100716,PREZENT,PREZENT,PREZENT,"6,8","8,4",8,...,None,NU,None,NU,None,"6,8","8,4",8,"7,73","9,32"
7,681851,M,URBAN,1461100716,PREZENT,PREZENT,PREZENT,"6,65","9,4",9,...,None,NU,None,NU,None,"6,65","9,4",9,"8,35","8,95"
8,683051,M,URBAN,1461100716,PREZENT,PREZENT,PREZENT,"5,25",7,"6,7",...,None,NU,None,NU,None,"5,25",7,"6,7","6,31","8,45"
9,683651,M,URBAN,1461100716,PREZENT,PREZENT,PREZENT,"4,4",5,"7,1",...,None,NU,None,NU,None,"4,4",5,"7,1","5,5","7,81"


In [44]:
#creez o lista cu coloanele care nu sunt de interes pentru analiza
columns_to_drop = ['STATUS ROMANA',' STATUS LIMBA MATERNA','STATUS MATEMATICA','NOTA ROMANA','NOTA LIMBA MATERNA','NOTA MATEMATICA',
                   'CONTESTATIE ROMANA','NOTA CONTESTATIE ROMANA','CONTESTATIE LIMBA MATERNA','NOTA CONTESTATIE LB MATERNA',
                   'CONTESTATIE MATEMATICA','NOTA CONTESTATIE MATEMATICA','NOTA FINALA ROMANA','NOTA FINALA LB MATERNA','NOTA FINALA MATEMATICA',]

#sterg coloanele din DataFrame-ul creat
df_rezultate_evaluare_nationala_2023_team2.drop(columns=columns_to_drop, axis = 1, inplace = True)
df_rezultate_evaluare_nationala_2023_team2.head(10) 

,COD UNIC CANDIDAT,SEX,MEDIU,cod_SIIIR,MEDIA,MEDIA V-VIII
0,2176401,M,RURAL,1461100707,"7,7","9,22"
1,2551851,M,RURAL,1461100707,"5,18","8,12"
2,2551551,M,RURAL,1461100707,"3,65","8,52"
3,2554651,F,RURAL,1461100707,"6,45","9,25"
4,681001,F,URBAN,1461100716,"9,36","9,74"
5,681601,F,URBAN,1461100716,"6,61","8,49"
6,681651,M,URBAN,1461100716,"7,73","9,32"
7,681851,M,URBAN,1461100716,"8,35","8,95"
8,683051,M,URBAN,1461100716,"6,31","8,45"
9,683651,M,URBAN,1461100716,"5,5","7,81"


In [45]:
#verific tipul coloanelor
column_dtypes = df_rezultate_evaluare_nationala_2023_team2.dtypes
column_dtypes

COD UNIC CANDIDAT     int64
SEX                  object
MEDIU                object
cod_SIIIR             int64
MEDIA                object
MEDIA V-VIII         object
dtype: object

In [46]:
#verific daca exista valori nule
df_rezultate_evaluare_nationala_2023_team2.isnull().sum()

COD UNIC CANDIDAT       0
SEX                     0
MEDIU                   0
cod_SIIIR               0
MEDIA                7537
MEDIA V-VIII            0
dtype: int64

In [47]:
#verific daca exista valori lipsa
df_rezultate_evaluare_nationala_2023_team2.isna().sum()

COD UNIC CANDIDAT       0
SEX                     0
MEDIU                   0
cod_SIIIR               0
MEDIA                7537
MEDIA V-VIII            0
dtype: int64

In [48]:
#sterg randurile care contin valori nule pe coloana 'MEDIA'
df_rezultate_evaluare_nationala_2023_team2['MEDIA'].dropna(inplace=True)
df_rezultate_evaluare_nationala_2023_team2 = df_rezultate_evaluare_nationala_2023_team2[df_rezultate_evaluare_nationala_2023_team2['MEDIA'].notna()]

In [51]:
#verific daca exista valori nule dupa indepartarea randurilor de la pasul anterior
df_rezultate_evaluare_nationala_2023_team2.isnull().sum()
df_rezultate_evaluare_nationala_2023_team2.isna().sum()

COD UNIC CANDIDAT    0
SEX                  0
MEDIU                0
cod_SIIIR            0
MEDIA                0
MEDIA V-VIII         0
dtype: int64

In [52]:
#inlocuiesc ',' cu '.' pentru a putea transforma in continuare tipul datelor
df_rezultate_evaluare_nationala_2023_team2['MEDIA'] = df_rezultate_evaluare_nationala_2023_team2['MEDIA'].str.replace(',','.') 
df_rezultate_evaluare_nationala_2023_team2.head(10)

,COD UNIC CANDIDAT,SEX,MEDIU,cod_SIIIR,MEDIA,MEDIA V-VIII
0,2176401,M,RURAL,1461100707,7.7,"9,22"
1,2551851,M,RURAL,1461100707,5.18,"8,12"
2,2551551,M,RURAL,1461100707,3.65,"8,52"
3,2554651,F,RURAL,1461100707,6.45,"9,25"
4,681001,F,URBAN,1461100716,9.36,"9,74"
5,681601,F,URBAN,1461100716,6.61,"8,49"
6,681651,M,URBAN,1461100716,7.73,"9,32"
7,681851,M,URBAN,1461100716,8.35,"8,95"
8,683051,M,URBAN,1461100716,6.31,"8,45"
9,683651,M,URBAN,1461100716,5.5,"7,81"


In [53]:
#convertesc in tip float pentru a avea date de tip numeric pentru analiza
df_rezultate_evaluare_nationala_2023_team2['MEDIA'] = df_rezultate_evaluare_nationala_2023_team2['MEDIA'].astype(float) 
df_rezultate_evaluare_nationala_2023_team2.head(10)

,COD UNIC CANDIDAT,SEX,MEDIU,cod_SIIIR,MEDIA,MEDIA V-VIII
0,2176401,M,RURAL,1461100707,7.70,"9,22"
1,2551851,M,RURAL,1461100707,5.18,"8,12"
2,2551551,M,RURAL,1461100707,3.65,"8,52"
3,2554651,F,RURAL,1461100707,6.45,"9,25"
4,681001,F,URBAN,1461100716,9.36,"9,74"
5,681601,F,URBAN,1461100716,6.61,"8,49"
6,681651,M,URBAN,1461100716,7.73,"9,32"
7,681851,M,URBAN,1461100716,8.35,"8,95"
8,683051,M,URBAN,1461100716,6.31,"8,45"
9,683651,M,URBAN,1461100716,5.50,"7,81"


In [54]:
#inlocuiesc ',' cu '.' pentru a putea transforma in continuare tipul datelor
df_rezultate_evaluare_nationala_2023_team2['MEDIA V-VIII'] = df_rezultate_evaluare_nationala_2023_team2['MEDIA V-VIII'].str.replace(',','.')
df_rezultate_evaluare_nationala_2023_team2.head(10)

,COD UNIC CANDIDAT,SEX,MEDIU,cod_SIIIR,MEDIA,MEDIA V-VIII
0,2176401,M,RURAL,1461100707,7.70,9.22
1,2551851,M,RURAL,1461100707,5.18,8.12
2,2551551,M,RURAL,1461100707,3.65,8.52
3,2554651,F,RURAL,1461100707,6.45,9.25
4,681001,F,URBAN,1461100716,9.36,9.74
5,681601,F,URBAN,1461100716,6.61,8.49
6,681651,M,URBAN,1461100716,7.73,9.32
7,681851,M,URBAN,1461100716,8.35,8.95
8,683051,M,URBAN,1461100716,6.31,8.45
9,683651,M,URBAN,1461100716,5.50,7.81


In [55]:
#convertesc in tip float pentru a avea date de tip numeric pentru analiza
df_rezultate_evaluare_nationala_2023_team2['MEDIA V-VIII'] = df_rezultate_evaluare_nationala_2023_team2['MEDIA V-VIII'].astype(float)
df_rezultate_evaluare_nationala_2023_team2.head(10)

,COD UNIC CANDIDAT,SEX,MEDIU,cod_SIIIR,MEDIA,MEDIA V-VIII
0,2176401,M,RURAL,1461100707,7.70,9.22
1,2551851,M,RURAL,1461100707,5.18,8.12
2,2551551,M,RURAL,1461100707,3.65,8.52
3,2554651,F,RURAL,1461100707,6.45,9.25
4,681001,F,URBAN,1461100716,9.36,9.74
5,681601,F,URBAN,1461100716,6.61,8.49
6,681651,M,URBAN,1461100716,7.73,9.32
7,681851,M,URBAN,1461100716,8.35,8.95
8,683051,M,URBAN,1461100716,6.31,8.45
9,683651,M,URBAN,1461100716,5.50,7.81


In [56]:
#verific tipul coloanelor
column_dtypes = df_rezultate_evaluare_nationala_2023_team2.dtypes
column_dtypes

COD UNIC CANDIDAT      int64
SEX                   object
MEDIU                 object
cod_SIIIR              int64
MEDIA                float64
MEDIA V-VIII         float64
dtype: object

### Preluarea tabelului cu informatii despre indicele de dezvoltare umana

In [57]:
#creez un query pentru import
query = 'SELECT * FROM lhdi_cluj_team2'
cursor.execute(query)
lhdi_cluj_team2 = cursor.fetchall()
lhdi_cluj_team2

[(54975, 'CLUJ', '80,1', 5, '80,5', '98,5', '26,9', 32367, 5, 'CLUJ-NAPOCA', 54984, 'CLUJ_CLUJ-NAPOCA'),
 (57706, 'CLUJ', '80', 5, '74,1', '127,6', '38,4', 35148, 5, 'FLORESTI', 57715, 'CLUJ_FLORESTI'),
 (57706, 'CLUJ', '80', 5, '74,1', '127,6', '38,4', 35148, 5, 'LUNA DE SUS', 57724, 'CLUJ_LUNA DE SUS'),
 (57706, 'CLUJ', '80', 5, '74,1', '127,6', '38,4', 35148, 5, 'TAUTI', 57733, 'CLUJ_TAUTI'),
 (58259, 'CLUJ', '77', 5, '86,4', '75,2', '34,5', 4454, 5, 'JUCU DE SUS', 58268, 'CLUJ_JUCU DE SUS'),
 (58259, 'CLUJ', '77', 5, '86,4', '75,2', '34,5', 4454, 5, 'GADALIN', 58277, 'CLUJ_GADALIN'),
 (58259, 'CLUJ', '77', 5, '86,4', '75,2', '34,5', 4454, 5, 'JUC-HERGHELIE', 58286, 'CLUJ_JUC-HERGHELIE'),
 (58259, 'CLUJ', '77', 5, '86,4', '75,2', '34,5', 4454, 5, 'JUCU DE MIJLOC', 58295, 'CLUJ_JUCU DE MIJLOC'),
 (58259, 'CLUJ', '77', 5, '86,4', '75,2', '34,5', 4454, 5, 'VISEA', 58302, 'CLUJ_VISEA'),
 (55687, 'CLUJ', '76,6', 5, '81,8', '86,6', '41,1', 13217, 5, 'APAHIDA', 55696, 'CLUJ_APAHIDA'),
 (55

In [58]:
#verific coloanele
columns_lhdi_cluj_team2 = [row[0] for row in cursor.description]
columns_lhdi_cluj_team2

['SIRUTA',
 'county',
 'LHDI2018',
 'IDUL2018quintile',
 'hcapmat18',
 'hconectint18',
 'hrsm18',
 'POPD2018',
 'Cultarea18',
 'locality',
 'SIRINF',
 'jud_loc']

In [59]:
#import datele intr-un DataFrame
result_dict_lhdi_cluj_team2 = {header: values for header, values in zip(columns_lhdi_cluj_team2, zip(*lhdi_cluj_team2))}
df_rezultate_lhdi_cluj_team2 = pd.DataFrame(result_dict_lhdi_cluj_team2)
df_rezultate_lhdi_cluj_team2.head(10)

,SIRUTA,county,LHDI2018,IDUL2018quintile,hcapmat18,hconectint18,hrsm18,POPD2018,Cultarea18,locality,SIRINF,jud_loc
0,54975,CLUJ,"80,1",5,"80,5","98,5","26,9",32367,5,CLUJ-NAPOCA,54984,CLUJ_CLUJ-NAPOCA
1,57706,CLUJ,80,5,"74,1","127,6","38,4",35148,5,FLORESTI,57715,CLUJ_FLORESTI
2,57706,CLUJ,80,5,"74,1","127,6","38,4",35148,5,LUNA DE SUS,57724,CLUJ_LUNA DE SUS
3,57706,CLUJ,80,5,"74,1","127,6","38,4",35148,5,TAUTI,57733,CLUJ_TAUTI
4,58259,CLUJ,77,5,"86,4","75,2","34,5",4454,5,JUCU DE SUS,58268,CLUJ_JUCU DE SUS
5,58259,CLUJ,77,5,"86,4","75,2","34,5",4454,5,GADALIN,58277,CLUJ_GADALIN
6,58259,CLUJ,77,5,"86,4","75,2","34,5",4454,5,JUC-HERGHELIE,58286,CLUJ_JUC-HERGHELIE
7,58259,CLUJ,77,5,"86,4","75,2","34,5",4454,5,JUCU DE MIJLOC,58295,CLUJ_JUCU DE MIJLOC
8,58259,CLUJ,77,5,"86,4","75,2","34,5",4454,5,VISEA,58302,CLUJ_VISEA
9,55687,CLUJ,"76,6",5,"81,8","86,6","41,1",13217,5,APAHIDA,55696,CLUJ_APAHIDA


In [60]:
#observ ca sunt coloane care se repeta si le sterg
#sterg coloana 'Cultarea18' deoarece contine o singura valoare
columns_to_drop1 = ['county', 'Cultarea18', 'locality']
df_rezultate_lhdi_cluj_team2.drop(columns=columns_to_drop1, axis = 1, inplace = True)
df_rezultate_lhdi_cluj_team2.head(10)

,SIRUTA,LHDI2018,IDUL2018quintile,hcapmat18,hconectint18,hrsm18,POPD2018,SIRINF,jud_loc
0,54975,"80,1",5,"80,5","98,5","26,9",32367,54984,CLUJ_CLUJ-NAPOCA
1,57706,80,5,"74,1","127,6","38,4",35148,57715,CLUJ_FLORESTI
2,57706,80,5,"74,1","127,6","38,4",35148,57724,CLUJ_LUNA DE SUS
3,57706,80,5,"74,1","127,6","38,4",35148,57733,CLUJ_TAUTI
4,58259,77,5,"86,4","75,2","34,5",4454,58268,CLUJ_JUCU DE SUS
5,58259,77,5,"86,4","75,2","34,5",4454,58277,CLUJ_GADALIN
6,58259,77,5,"86,4","75,2","34,5",4454,58286,CLUJ_JUC-HERGHELIE
7,58259,77,5,"86,4","75,2","34,5",4454,58295,CLUJ_JUCU DE MIJLOC
8,58259,77,5,"86,4","75,2","34,5",4454,58302,CLUJ_VISEA
9,55687,"76,6",5,"81,8","86,6","41,1",13217,55696,CLUJ_APAHIDA


In [61]:
#verific tipul coloanelor
column_dtypes = df_rezultate_lhdi_cluj_team2.dtypes
column_dtypes

SIRUTA               int64
LHDI2018            object
IDUL2018quintile     int64
hcapmat18           object
hconectint18        object
hrsm18              object
POPD2018             int64
SIRINF               int64
jud_loc             object
dtype: object

In [62]:
#verific daca exista valori nule
df_rezultate_lhdi_cluj_team2.isnull().sum()

SIRUTA              0
LHDI2018            0
IDUL2018quintile    0
hcapmat18           0
hconectint18        0
hrsm18              0
POPD2018            0
SIRINF              0
jud_loc             0
dtype: int64

In [63]:
#verific daca exista valori lipsa
df_rezultate_lhdi_cluj_team2.isna().sum()

SIRUTA              0
LHDI2018            0
IDUL2018quintile    0
hcapmat18           0
hconectint18        0
hrsm18              0
POPD2018            0
SIRINF              0
jud_loc             0
dtype: int64

In [64]:
#inlocuiesc ',' cu '.' pentru a putea transforma in continuare tipul datelor
df_rezultate_lhdi_cluj_team2['LHDI2018'] = df_rezultate_lhdi_cluj_team2['LHDI2018'].str.replace(',','.')
df_rezultate_lhdi_cluj_team2.head(10)

,SIRUTA,LHDI2018,IDUL2018quintile,hcapmat18,hconectint18,hrsm18,POPD2018,SIRINF,jud_loc
0,54975,80.1,5,"80,5","98,5","26,9",32367,54984,CLUJ_CLUJ-NAPOCA
1,57706,80,5,"74,1","127,6","38,4",35148,57715,CLUJ_FLORESTI
2,57706,80,5,"74,1","127,6","38,4",35148,57724,CLUJ_LUNA DE SUS
3,57706,80,5,"74,1","127,6","38,4",35148,57733,CLUJ_TAUTI
4,58259,77,5,"86,4","75,2","34,5",4454,58268,CLUJ_JUCU DE SUS
5,58259,77,5,"86,4","75,2","34,5",4454,58277,CLUJ_GADALIN
6,58259,77,5,"86,4","75,2","34,5",4454,58286,CLUJ_JUC-HERGHELIE
7,58259,77,5,"86,4","75,2","34,5",4454,58295,CLUJ_JUCU DE MIJLOC
8,58259,77,5,"86,4","75,2","34,5",4454,58302,CLUJ_VISEA
9,55687,76.6,5,"81,8","86,6","41,1",13217,55696,CLUJ_APAHIDA


In [65]:
#convertesc in tip float pentru a avea date de tip numeric pentru analiza
df_rezultate_lhdi_cluj_team2['LHDI2018'] = df_rezultate_lhdi_cluj_team2['LHDI2018'].astype(float)
df_rezultate_lhdi_cluj_team2.head(10)

,SIRUTA,LHDI2018,IDUL2018quintile,hcapmat18,hconectint18,hrsm18,POPD2018,SIRINF,jud_loc
0,54975,80.1,5,"80,5","98,5","26,9",32367,54984,CLUJ_CLUJ-NAPOCA
1,57706,80.0,5,"74,1","127,6","38,4",35148,57715,CLUJ_FLORESTI
2,57706,80.0,5,"74,1","127,6","38,4",35148,57724,CLUJ_LUNA DE SUS
3,57706,80.0,5,"74,1","127,6","38,4",35148,57733,CLUJ_TAUTI
4,58259,77.0,5,"86,4","75,2","34,5",4454,58268,CLUJ_JUCU DE SUS
5,58259,77.0,5,"86,4","75,2","34,5",4454,58277,CLUJ_GADALIN
6,58259,77.0,5,"86,4","75,2","34,5",4454,58286,CLUJ_JUC-HERGHELIE
7,58259,77.0,5,"86,4","75,2","34,5",4454,58295,CLUJ_JUCU DE MIJLOC
8,58259,77.0,5,"86,4","75,2","34,5",4454,58302,CLUJ_VISEA
9,55687,76.6,5,"81,8","86,6","41,1",13217,55696,CLUJ_APAHIDA


In [66]:
#inlocuiesc ',' cu '.' pentru a putea transforma in continuare tipul datelor
df_rezultate_lhdi_cluj_team2['hcapmat18'] = df_rezultate_lhdi_cluj_team2['hcapmat18'].str.replace(',','.')
df_rezultate_lhdi_cluj_team2.head(10)

,SIRUTA,LHDI2018,IDUL2018quintile,hcapmat18,hconectint18,hrsm18,POPD2018,SIRINF,jud_loc
0,54975,80.1,5,80.5,"98,5","26,9",32367,54984,CLUJ_CLUJ-NAPOCA
1,57706,80.0,5,74.1,"127,6","38,4",35148,57715,CLUJ_FLORESTI
2,57706,80.0,5,74.1,"127,6","38,4",35148,57724,CLUJ_LUNA DE SUS
3,57706,80.0,5,74.1,"127,6","38,4",35148,57733,CLUJ_TAUTI
4,58259,77.0,5,86.4,"75,2","34,5",4454,58268,CLUJ_JUCU DE SUS
5,58259,77.0,5,86.4,"75,2","34,5",4454,58277,CLUJ_GADALIN
6,58259,77.0,5,86.4,"75,2","34,5",4454,58286,CLUJ_JUC-HERGHELIE
7,58259,77.0,5,86.4,"75,2","34,5",4454,58295,CLUJ_JUCU DE MIJLOC
8,58259,77.0,5,86.4,"75,2","34,5",4454,58302,CLUJ_VISEA
9,55687,76.6,5,81.8,"86,6","41,1",13217,55696,CLUJ_APAHIDA


In [67]:
#convertesc in tip float pentru a avea date de tip numeric pentru analiza
df_rezultate_lhdi_cluj_team2['hcapmat18'] = df_rezultate_lhdi_cluj_team2['hcapmat18'].astype(float)
df_rezultate_lhdi_cluj_team2.head(10)

,SIRUTA,LHDI2018,IDUL2018quintile,hcapmat18,hconectint18,hrsm18,POPD2018,SIRINF,jud_loc
0,54975,80.1,5,80.5,"98,5","26,9",32367,54984,CLUJ_CLUJ-NAPOCA
1,57706,80.0,5,74.1,"127,6","38,4",35148,57715,CLUJ_FLORESTI
2,57706,80.0,5,74.1,"127,6","38,4",35148,57724,CLUJ_LUNA DE SUS
3,57706,80.0,5,74.1,"127,6","38,4",35148,57733,CLUJ_TAUTI
4,58259,77.0,5,86.4,"75,2","34,5",4454,58268,CLUJ_JUCU DE SUS
5,58259,77.0,5,86.4,"75,2","34,5",4454,58277,CLUJ_GADALIN
6,58259,77.0,5,86.4,"75,2","34,5",4454,58286,CLUJ_JUC-HERGHELIE
7,58259,77.0,5,86.4,"75,2","34,5",4454,58295,CLUJ_JUCU DE MIJLOC
8,58259,77.0,5,86.4,"75,2","34,5",4454,58302,CLUJ_VISEA
9,55687,76.6,5,81.8,"86,6","41,1",13217,55696,CLUJ_APAHIDA


In [68]:
#inlocuiesc ',' cu '.' pentru a putea transforma in continuare tipul datelor
df_rezultate_lhdi_cluj_team2['hconectint18'] = df_rezultate_lhdi_cluj_team2['hconectint18'].str.replace(',','.')
df_rezultate_lhdi_cluj_team2.head(10)

,SIRUTA,LHDI2018,IDUL2018quintile,hcapmat18,hconectint18,hrsm18,POPD2018,SIRINF,jud_loc
0,54975,80.1,5,80.5,98.5,"26,9",32367,54984,CLUJ_CLUJ-NAPOCA
1,57706,80.0,5,74.1,127.6,"38,4",35148,57715,CLUJ_FLORESTI
2,57706,80.0,5,74.1,127.6,"38,4",35148,57724,CLUJ_LUNA DE SUS
3,57706,80.0,5,74.1,127.6,"38,4",35148,57733,CLUJ_TAUTI
4,58259,77.0,5,86.4,75.2,"34,5",4454,58268,CLUJ_JUCU DE SUS
5,58259,77.0,5,86.4,75.2,"34,5",4454,58277,CLUJ_GADALIN
6,58259,77.0,5,86.4,75.2,"34,5",4454,58286,CLUJ_JUC-HERGHELIE
7,58259,77.0,5,86.4,75.2,"34,5",4454,58295,CLUJ_JUCU DE MIJLOC
8,58259,77.0,5,86.4,75.2,"34,5",4454,58302,CLUJ_VISEA
9,55687,76.6,5,81.8,86.6,"41,1",13217,55696,CLUJ_APAHIDA


In [69]:
#convertesc in tip float pentru a avea date de tip numeric pentru analiza
df_rezultate_lhdi_cluj_team2['hconectint18'] = df_rezultate_lhdi_cluj_team2['hconectint18'].astype(float)
df_rezultate_lhdi_cluj_team2.head(10)

,SIRUTA,LHDI2018,IDUL2018quintile,hcapmat18,hconectint18,hrsm18,POPD2018,SIRINF,jud_loc
0,54975,80.1,5,80.5,98.5,"26,9",32367,54984,CLUJ_CLUJ-NAPOCA
1,57706,80.0,5,74.1,127.6,"38,4",35148,57715,CLUJ_FLORESTI
2,57706,80.0,5,74.1,127.6,"38,4",35148,57724,CLUJ_LUNA DE SUS
3,57706,80.0,5,74.1,127.6,"38,4",35148,57733,CLUJ_TAUTI
4,58259,77.0,5,86.4,75.2,"34,5",4454,58268,CLUJ_JUCU DE SUS
5,58259,77.0,5,86.4,75.2,"34,5",4454,58277,CLUJ_GADALIN
6,58259,77.0,5,86.4,75.2,"34,5",4454,58286,CLUJ_JUC-HERGHELIE
7,58259,77.0,5,86.4,75.2,"34,5",4454,58295,CLUJ_JUCU DE MIJLOC
8,58259,77.0,5,86.4,75.2,"34,5",4454,58302,CLUJ_VISEA
9,55687,76.6,5,81.8,86.6,"41,1",13217,55696,CLUJ_APAHIDA


In [70]:
#inlocuiesc ',' cu '.' pentru a putea transforma in continuare tipul datelor
df_rezultate_lhdi_cluj_team2['hrsm18'] = df_rezultate_lhdi_cluj_team2['hrsm18'].str.replace(',','.')
df_rezultate_lhdi_cluj_team2.head(10)

,SIRUTA,LHDI2018,IDUL2018quintile,hcapmat18,hconectint18,hrsm18,POPD2018,SIRINF,jud_loc
0,54975,80.1,5,80.5,98.5,26.9,32367,54984,CLUJ_CLUJ-NAPOCA
1,57706,80.0,5,74.1,127.6,38.4,35148,57715,CLUJ_FLORESTI
2,57706,80.0,5,74.1,127.6,38.4,35148,57724,CLUJ_LUNA DE SUS
3,57706,80.0,5,74.1,127.6,38.4,35148,57733,CLUJ_TAUTI
4,58259,77.0,5,86.4,75.2,34.5,4454,58268,CLUJ_JUCU DE SUS
5,58259,77.0,5,86.4,75.2,34.5,4454,58277,CLUJ_GADALIN
6,58259,77.0,5,86.4,75.2,34.5,4454,58286,CLUJ_JUC-HERGHELIE
7,58259,77.0,5,86.4,75.2,34.5,4454,58295,CLUJ_JUCU DE MIJLOC
8,58259,77.0,5,86.4,75.2,34.5,4454,58302,CLUJ_VISEA
9,55687,76.6,5,81.8,86.6,41.1,13217,55696,CLUJ_APAHIDA


In [71]:
#convertesc in tip float pentru a avea date de tip numeric pentru analiza
df_rezultate_lhdi_cluj_team2['hrsm18'] = df_rezultate_lhdi_cluj_team2['hrsm18'].astype(float)
df_rezultate_lhdi_cluj_team2.head(10)

,SIRUTA,LHDI2018,IDUL2018quintile,hcapmat18,hconectint18,hrsm18,POPD2018,SIRINF,jud_loc
0,54975,80.1,5,80.5,98.5,26.9,32367,54984,CLUJ_CLUJ-NAPOCA
1,57706,80.0,5,74.1,127.6,38.4,35148,57715,CLUJ_FLORESTI
2,57706,80.0,5,74.1,127.6,38.4,35148,57724,CLUJ_LUNA DE SUS
3,57706,80.0,5,74.1,127.6,38.4,35148,57733,CLUJ_TAUTI
4,58259,77.0,5,86.4,75.2,34.5,4454,58268,CLUJ_JUCU DE SUS
5,58259,77.0,5,86.4,75.2,34.5,4454,58277,CLUJ_GADALIN
6,58259,77.0,5,86.4,75.2,34.5,4454,58286,CLUJ_JUC-HERGHELIE
7,58259,77.0,5,86.4,75.2,34.5,4454,58295,CLUJ_JUCU DE MIJLOC
8,58259,77.0,5,86.4,75.2,34.5,4454,58302,CLUJ_VISEA
9,55687,76.6,5,81.8,86.6,41.1,13217,55696,CLUJ_APAHIDA


In [72]:
#verific tipul coloanelor
column_dtypes = df_rezultate_lhdi_cluj_team2.dtypes
column_dtypes

SIRUTA                int64
LHDI2018            float64
IDUL2018quintile      int64
hcapmat18           float64
hconectint18        float64
hrsm18              float64
POPD2018              int64
SIRINF                int64
jud_loc              object
dtype: object

## Unesc tabelele pentru a obtine o baza de date finala

In [73]:
#verific daca exista duplicate in primul tabel
print(df_school_details_cj_team2.duplicated().sum())

0


In [74]:
#verific daca exista duplicate in al doilea tabel
print(df_school_buildings_team2.duplicated().sum())

73


In [ ]:
#Unesc primele doua tabele prin coloana 'idSchool' si obtin datele comune dintre cele doua

In [75]:
school_details_buildings_team2 = pd.merge(df_school_details_cj_team2, df_school_buildings_team2, on = 'idSchool', how = 'inner')

In [76]:
school_details_buildings_team2

,idSchool,siruesCode,shortName,county,locality,studyFormationsCount,studentsCount,personnelCount,buildingDescription,generalState,buildingMaterial,roofMaterial,roofCondition,joineryCondition,windowJoinery,doorJoinery
0,11266864,121121007,CLUBUL COPIILOR DEJ,CLUJ,DEJ,0,0,0,SALA GIMNASTICA,1,CADRU BETON,TIGLA METALICA,1,1,PVC,PVC
1,11266872,121266467,"CLUBUL SCOLAR SPORTIV ""VIITORUL"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,0,0,33,SALA LUPTE,2,CARAMIDA,TIGLA CERAMICA,3,2,PVC,PVC
2,11266914,121181148,"COLEGIUL ECONOMIC ""IULIAN POP"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,15,529,52,CLADIREA VECHE 1,5,CARAMIDA,TIGLA CERAMICA,5,5,LEMN,LEMN
3,11266914,121181148,"COLEGIUL ECONOMIC ""IULIAN POP"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,15,529,52,CLADIREA VECHE,2,CARAMIDA,TIGLA CERAMICA,5,2,PVC,PVC
4,11266914,121181148,"COLEGIUL ECONOMIC ""IULIAN POP"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,15,529,52,CAMIN,1,CADRU BETON,TABLA,1,1,PVC,LEMN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,11267124,120849509,"SCOALA PROFESIONALA SPECIALA ""SAMUS"" CLUJ - NA...",CLUJ,CLUJ-NAPOCA,37,245,123,CLADIRE SPALATORIE,1,CARAMIDA,TIGLA CERAMICA,1,1,PVC,PVC
433,11267124,120849509,"SCOALA PROFESIONALA SPECIALA ""SAMUS"" CLUJ - NA...",CLUJ,CLUJ-NAPOCA,37,245,123,CLADIRE CASA POARTA,5,CARAMIDA,TIGLA CERAMICA,5,5,LEMN,LEMN
434,11267124,120849509,"SCOALA PROFESIONALA SPECIALA ""SAMUS"" CLUJ - NA...",CLUJ,CLUJ-NAPOCA,37,245,123,CLADIRE INTERNAT,5,CADRU BETON,TIGLA METALICA,2,1,PVC,LEMN
435,11267124,120849509,"SCOALA PROFESIONALA SPECIALA ""SAMUS"" CLUJ - NA...",CLUJ,CLUJ-NAPOCA,37,245,123,CLADIRE CENTRALA TERMICA,5,CARAMIDA,TIGLA CERAMICA,5,5,OȚEL,OȚEL


In [77]:
#verific daca in coloana 'shortname' exista duplicate
duplicates_mask = school_details_buildings_team2['shortName'].duplicated(keep='first')
duplicates_mask

0      False
1      False
2      False
3       True
4       True
       ...  
432    False
433     True
434     True
435     True
436     True
Name: shortName, Length: 437, dtype: bool

In [ ]:
##Deoarece marea majoritate a scolilor sunt compuse din mai multe cladiri,
##am decis sa pastram prima inregistrare in fiecare caz
##pentru a studia datele despre starea cladirilor

In [78]:
#sterg duplicatele pastrand prima inregistrare din coloana 'shortname'
school_details_buildings_team2 = school_details_buildings_team2[~school_details_buildings_team2['shortName'].duplicated(keep='first')]

In [79]:
school_details_buildings_team2.head(5)

,idSchool,siruesCode,shortName,county,locality,studyFormationsCount,studentsCount,personnelCount,buildingDescription,generalState,buildingMaterial,roofMaterial,roofCondition,joineryCondition,windowJoinery,doorJoinery
0,11266864,121121007,CLUBUL COPIILOR DEJ,CLUJ,DEJ,0,0,0,SALA GIMNASTICA,1,CADRU BETON,TIGLA METALICA,1,1,PVC,PVC
1,11266872,121266467,"CLUBUL SCOLAR SPORTIV ""VIITORUL"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,0,0,33,SALA LUPTE,2,CARAMIDA,TIGLA CERAMICA,3,2,PVC,PVC
2,11266914,121181148,"COLEGIUL ECONOMIC ""IULIAN POP"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,15,529,52,CLADIREA VECHE 1,5,CARAMIDA,TIGLA CERAMICA,5,5,LEMN,LEMN
5,11266915,120844602,"COLEGIUL ""EMIL NEGRUTIU"" TURDA",CLUJ,TURDA,25,596,58,CORP B,2,CARAMIDA,TIGLA CERAMICA,5,5,PVC,PVC
8,11266917,120437849,"COLEGIUL NATIONAL ""EMIL RACOVITA"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,36,1071,88,C SALI DE CLASA PRIMAR BIBLIOTECA CABINET MEDICAL,1,CARAMIDA,TIGLA CERAMICA,1,1,MIXTA,MIXTA


In [80]:
#dupa filtrarea anterioara, coloana cu descrierea cladirii nu mai este necesara
#sterg coloana
school_details_buildings_team2 = school_details_buildings_team2.drop('buildingDescription', axis=1)
school_details_buildings_team2

,idSchool,siruesCode,shortName,county,locality,studyFormationsCount,studentsCount,personnelCount,generalState,buildingMaterial,roofMaterial,roofCondition,joineryCondition,windowJoinery,doorJoinery
0,11266864,121121007,CLUBUL COPIILOR DEJ,CLUJ,DEJ,0,0,0,1,CADRU BETON,TIGLA METALICA,1,1,PVC,PVC
1,11266872,121266467,"CLUBUL SCOLAR SPORTIV ""VIITORUL"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,0,0,33,2,CARAMIDA,TIGLA CERAMICA,3,2,PVC,PVC
2,11266914,121181148,"COLEGIUL ECONOMIC ""IULIAN POP"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,15,529,52,5,CARAMIDA,TIGLA CERAMICA,5,5,LEMN,LEMN
5,11266915,120844602,"COLEGIUL ""EMIL NEGRUTIU"" TURDA",CLUJ,TURDA,25,596,58,2,CARAMIDA,TIGLA CERAMICA,5,5,PVC,PVC
8,11266917,120437849,"COLEGIUL NATIONAL ""EMIL RACOVITA"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,36,1071,88,1,CARAMIDA,TIGLA CERAMICA,1,1,MIXTA,MIXTA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,11267356,120442076,"SCOALA PRIMARA VISTEA, COM. GARBAU",CLUJ,VIŞTEA,8,0,0,2,ALTE MATERIALE,TIGLA CERAMICA,4,4,PVC,PVC
425,11267358,<NA>,SCOALA PRIMARA WALDORF TURDA,CLUJ,TURDA,8,0,0,5,CARAMIDA,TIGLA CERAMICA,5,5,LEMN,LEMN
426,11267360,120440717,SCOALA PROFESIONALA POIANA TURDA,CLUJ,TURDA,26,411,39,1,CARAMIDA,TIGLA CERAMICA,4,4,PVC,PVC
429,11267345,120438829,"SCOALA PROFESIONALA RASCRUCI, COM. BONTIDA",CLUJ,RASCRUCI,23,250,42,2,CARAMIDA,TIGLA METALICA,1,2,PVC,PVC


In [81]:
#verific duplicatele
print(school_details_buildings_team2.duplicated().sum())

0


In [ ]:
##Unesc tabelul obtinut mai devreme cu tabelul siiir pe baza coloanei 'sirues'
##Avem nevoie de codul siiir pentru a putea aduce in contiuare tabelul cu notele de la evaluarea nationala

In [82]:
school_buildings_siiir_team2 = pd.merge(school_details_buildings_team2, df_sirues_siiir_team2, on = 'siruesCode', how = 'left')

In [83]:
school_buildings_siiir_team2

,idSchool,siruesCode,shortName,county,locality,studyFormationsCount,studentsCount,personnelCount,generalState,buildingMaterial,roofMaterial,roofCondition,joineryCondition,windowJoinery,doorJoinery,cod_SIIIR
0,11266864,121121007,CLUBUL COPIILOR DEJ,CLUJ,DEJ,0,0,0,1,CADRU BETON,TIGLA METALICA,1,1,PVC,PVC,1.252104e+09
1,11266872,121266467,"CLUBUL SCOLAR SPORTIV ""VIITORUL"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,0,0,33,2,CARAMIDA,TIGLA CERAMICA,3,2,PVC,PVC,1.281102e+09
2,11266914,121181148,"COLEGIUL ECONOMIC ""IULIAN POP"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,15,529,52,5,CARAMIDA,TIGLA CERAMICA,5,5,LEMN,LEMN,1.261103e+09
3,11266915,120844602,"COLEGIUL ""EMIL NEGRUTIU"" TURDA",CLUJ,TURDA,25,596,58,2,CARAMIDA,TIGLA CERAMICA,5,5,PVC,PVC,1.261102e+09
4,11266917,120437849,"COLEGIUL NATIONAL ""EMIL RACOVITA"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,36,1071,88,1,CARAMIDA,TIGLA CERAMICA,1,1,MIXTA,MIXTA,1.261105e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,11267356,120442076,"SCOALA PRIMARA VISTEA, COM. GARBAU",CLUJ,VIŞTEA,8,0,0,2,ALTE MATERIALE,TIGLA CERAMICA,4,4,PVC,PVC,1.262102e+09
252,11267358,<NA>,SCOALA PRIMARA WALDORF TURDA,CLUJ,TURDA,8,0,0,5,CARAMIDA,TIGLA CERAMICA,5,5,LEMN,LEMN,NaN
253,11267360,120440717,SCOALA PROFESIONALA POIANA TURDA,CLUJ,TURDA,26,411,39,1,CARAMIDA,TIGLA CERAMICA,4,4,PVC,PVC,1.261104e+09
254,11267345,120438829,"SCOALA PROFESIONALA RASCRUCI, COM. BONTIDA",CLUJ,RASCRUCI,23,250,42,2,CARAMIDA,TIGLA METALICA,1,2,PVC,PVC,1.261102e+09


In [84]:
#export tabelul obtinut intr-un csv pentru verificare
output_file_path = 'school_buildings_siiir_team2.csv'
school_buildings_siiir_team2.to_csv(output_file_path, index=False)

In [85]:
school_buildings_siiir_team2.isnull().sum()

idSchool                  0
siruesCode               57
shortName                 0
county                    0
locality                  0
studyFormationsCount      0
studentsCount             0
personnelCount            0
generalState              0
buildingMaterial          0
roofMaterial              0
roofCondition             0
joineryCondition          0
windowJoinery             0
doorJoinery               0
cod_SIIIR               105
dtype: int64

In [86]:
##Sterg randurile care au valori nule pe coloana siruesCode, deoarece acestea
##sunt gradinite, scoli primare sau alte scoli
##care nu sustin examenul de evaluare nationala

In [87]:
target_column = 'siruesCode'
school_buildings_siiir_team2.dropna(subset=[target_column], inplace=True)

In [88]:
school_buildings_siiir_team2

,idSchool,siruesCode,shortName,county,locality,studyFormationsCount,studentsCount,personnelCount,generalState,buildingMaterial,roofMaterial,roofCondition,joineryCondition,windowJoinery,doorJoinery,cod_SIIIR
0,11266864,121121007,CLUBUL COPIILOR DEJ,CLUJ,DEJ,0,0,0,1,CADRU BETON,TIGLA METALICA,1,1,PVC,PVC,1.252104e+09
1,11266872,121266467,"CLUBUL SCOLAR SPORTIV ""VIITORUL"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,0,0,33,2,CARAMIDA,TIGLA CERAMICA,3,2,PVC,PVC,1.281102e+09
2,11266914,121181148,"COLEGIUL ECONOMIC ""IULIAN POP"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,15,529,52,5,CARAMIDA,TIGLA CERAMICA,5,5,LEMN,LEMN,1.261103e+09
3,11266915,120844602,"COLEGIUL ""EMIL NEGRUTIU"" TURDA",CLUJ,TURDA,25,596,58,2,CARAMIDA,TIGLA CERAMICA,5,5,PVC,PVC,1.261102e+09
4,11266917,120437849,"COLEGIUL NATIONAL ""EMIL RACOVITA"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,36,1071,88,1,CARAMIDA,TIGLA CERAMICA,1,1,MIXTA,MIXTA,1.261105e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,11267355,120441242,"SCOALA PRIMARA VALENI, COM. CALATELE",CLUJ,VALENI,6,0,0,2,CARAMIDA,ALTE MATERIALE,1,1,PVC,LEMN,1.262101e+09
251,11267356,120442076,"SCOALA PRIMARA VISTEA, COM. GARBAU",CLUJ,VIŞTEA,8,0,0,2,ALTE MATERIALE,TIGLA CERAMICA,4,4,PVC,PVC,1.262102e+09
253,11267360,120440717,SCOALA PROFESIONALA POIANA TURDA,CLUJ,TURDA,26,411,39,1,CARAMIDA,TIGLA CERAMICA,4,4,PVC,PVC,1.261104e+09
254,11267345,120438829,"SCOALA PROFESIONALA RASCRUCI, COM. BONTIDA",CLUJ,RASCRUCI,23,250,42,2,CARAMIDA,TIGLA METALICA,1,2,PVC,PVC,1.261102e+09


In [89]:
#verific daca exista valori nule
school_buildings_siiir_team2.isnull().sum()

idSchool                 0
siruesCode               0
shortName                0
county                   0
locality                 0
studyFormationsCount     0
studentsCount            0
personnelCount           0
generalState             0
buildingMaterial         0
roofMaterial             0
roofCondition            0
joineryCondition         0
windowJoinery            0
doorJoinery              0
cod_SIIIR               48
dtype: int64

In [90]:
#concatenez coloanele 'county' si 'locality' pentru a putea obtine
#o coloana de legatura cu tabelul lhdi
#coloanele initiale pot fi utile in analiza ulterioara
school_buildings_siiir_team2['jud_loc'] = pd.concat([school_buildings_siiir_team2['county'], school_buildings_siiir_team2['locality']], axis=1).apply(lambda x: '_'.join(x), axis=1)

In [91]:
school_buildings_siiir_team2.head(5)

,idSchool,siruesCode,shortName,county,locality,studyFormationsCount,studentsCount,personnelCount,generalState,buildingMaterial,roofMaterial,roofCondition,joineryCondition,windowJoinery,doorJoinery,cod_SIIIR,jud_loc
0,11266864,121121007,CLUBUL COPIILOR DEJ,CLUJ,DEJ,0,0,0,1,CADRU BETON,TIGLA METALICA,1,1,PVC,PVC,1.252104e+09,CLUJ_DEJ
1,11266872,121266467,"CLUBUL SCOLAR SPORTIV ""VIITORUL"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,0,0,33,2,CARAMIDA,TIGLA CERAMICA,3,2,PVC,PVC,1.281102e+09,CLUJ_CLUJ-NAPOCA
2,11266914,121181148,"COLEGIUL ECONOMIC ""IULIAN POP"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,15,529,52,5,CARAMIDA,TIGLA CERAMICA,5,5,LEMN,LEMN,1.261103e+09,CLUJ_CLUJ-NAPOCA
3,11266915,120844602,"COLEGIUL ""EMIL NEGRUTIU"" TURDA",CLUJ,TURDA,25,596,58,2,CARAMIDA,TIGLA CERAMICA,5,5,PVC,PVC,1.261102e+09,CLUJ_TURDA
4,11266917,120437849,"COLEGIUL NATIONAL ""EMIL RACOVITA"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,36,1071,88,1,CARAMIDA,TIGLA CERAMICA,1,1,MIXTA,MIXTA,1.261105e+09,CLUJ_CLUJ-NAPOCA


In [ ]:
##Unesc tabelul obtinut mai devreme cu tabelul rezultate evaluare nationala pe baza coloanei 'cod_SIIR'
##si obtin datele comune dintre cele doua
##Valorile nule obtinute anterior pentru coloana 'cod_SIIR' sunt ignorate
##deoarece folosim inner join

In [92]:
school_buildings_siiir_en_team2 = pd.merge(school_buildings_siiir_team2, df_rezultate_evaluare_nationala_2023_team2, on = 'cod_SIIIR', how = 'inner')

In [93]:
school_buildings_siiir_en_team2.head(5)

,idSchool,siruesCode,shortName,county,locality,studyFormationsCount,studentsCount,personnelCount,generalState,buildingMaterial,...,joineryCondition,windowJoinery,doorJoinery,cod_SIIIR,jud_loc,COD UNIC CANDIDAT,SEX,MEDIU,MEDIA,MEDIA V-VIII
0,11266917,120437849,"COLEGIUL NATIONAL ""EMIL RACOVITA"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,36,1071,88,1,CARAMIDA,...,1,MIXTA,MIXTA,1.261105e+09,CLUJ_CLUJ-NAPOCA,10107762,M,URBAN,9.85,9.89
1,11266917,120437849,"COLEGIUL NATIONAL ""EMIL RACOVITA"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,36,1071,88,1,CARAMIDA,...,1,MIXTA,MIXTA,1.261105e+09,CLUJ_CLUJ-NAPOCA,10302839,F,URBAN,9.67,9.67
2,11266917,120437849,"COLEGIUL NATIONAL ""EMIL RACOVITA"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,36,1071,88,1,CARAMIDA,...,1,MIXTA,MIXTA,1.261105e+09,CLUJ_CLUJ-NAPOCA,10107674,F,URBAN,9.40,9.86
3,11266917,120437849,"COLEGIUL NATIONAL ""EMIL RACOVITA"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,36,1071,88,1,CARAMIDA,...,1,MIXTA,MIXTA,1.261105e+09,CLUJ_CLUJ-NAPOCA,10283941,F,URBAN,9.00,9.21
4,11266917,120437849,"COLEGIUL NATIONAL ""EMIL RACOVITA"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,36,1071,88,1,CARAMIDA,...,1,MIXTA,MIXTA,1.261105e+09,CLUJ_CLUJ-NAPOCA,10311207,M,URBAN,9.95,9.85


In [94]:
#verific daca exista valori nule
school_buildings_siiir_en_team2.isnull().sum()

idSchool                0
siruesCode              0
shortName               0
county                  0
locality                0
studyFormationsCount    0
studentsCount           0
personnelCount          0
generalState            0
buildingMaterial        0
roofMaterial            0
roofCondition           0
joineryCondition        0
windowJoinery           0
doorJoinery             0
cod_SIIIR               0
jud_loc                 0
COD UNIC CANDIDAT       0
SEX                     0
MEDIU                   0
MEDIA                   0
MEDIA V-VIII            0
dtype: int64

In [95]:
#verific daca exista valori duplicate
print(school_buildings_siiir_en_team2.duplicated().sum())

0


In [ ]:
##Unesc tabelul obtinut mai devreme cu tabelul lhdi pe baza coloanei 'jud_cod'
##si obtin datele comune dintre cele doua

In [96]:
school_buildings_siiir_en_lhdi_team2 = pd.merge(school_buildings_siiir_en_team2, df_rezultate_lhdi_cluj_team2, on = 'jud_loc', how = 'inner')

In [97]:
school_buildings_siiir_en_lhdi_team2

,idSchool,siruesCode,shortName,county,locality,studyFormationsCount,studentsCount,personnelCount,generalState,buildingMaterial,...,MEDIA,MEDIA V-VIII,SIRUTA,LHDI2018,IDUL2018quintile,hcapmat18,hconectint18,hrsm18,POPD2018,SIRINF
0,11266917,120437849,"COLEGIUL NATIONAL ""EMIL RACOVITA"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,36,1071,88,1,CARAMIDA,...,9.85,9.89,54975,80.1,5,80.5,98.5,26.9,32367,54984
1,11266917,120437849,"COLEGIUL NATIONAL ""EMIL RACOVITA"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,36,1071,88,1,CARAMIDA,...,9.67,9.67,54975,80.1,5,80.5,98.5,26.9,32367,54984
2,11266917,120437849,"COLEGIUL NATIONAL ""EMIL RACOVITA"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,36,1071,88,1,CARAMIDA,...,9.40,9.86,54975,80.1,5,80.5,98.5,26.9,32367,54984
3,11266917,120437849,"COLEGIUL NATIONAL ""EMIL RACOVITA"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,36,1071,88,1,CARAMIDA,...,9.00,9.21,54975,80.1,5,80.5,98.5,26.9,32367,54984
4,11266917,120437849,"COLEGIUL NATIONAL ""EMIL RACOVITA"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,36,1071,88,1,CARAMIDA,...,9.95,9.85,54975,80.1,5,80.5,98.5,26.9,32367,54984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3421,11267345,120438829,"SCOALA PROFESIONALA RASCRUCI, COM. BONTIDA",CLUJ,RASCRUCI,23,250,42,2,CARAMIDA,...,6.70,9.83,56210,59.4,5,62.9,62.0,52.9,4885,56247
3422,11267345,120438829,"SCOALA PROFESIONALA RASCRUCI, COM. BONTIDA",CLUJ,RASCRUCI,23,250,42,2,CARAMIDA,...,8.62,9.82,56210,59.4,5,62.9,62.0,52.9,4885,56247
3423,11267345,120438829,"SCOALA PROFESIONALA RASCRUCI, COM. BONTIDA",CLUJ,RASCRUCI,23,250,42,2,CARAMIDA,...,6.37,9.38,56210,59.4,5,62.9,62.0,52.9,4885,56247
3424,11267345,120438829,"SCOALA PROFESIONALA RASCRUCI, COM. BONTIDA",CLUJ,RASCRUCI,23,250,42,2,CARAMIDA,...,7.65,9.57,56210,59.4,5,62.9,62.0,52.9,4885,56247


In [98]:
#verific daca exista valori nule
school_buildings_siiir_en_lhdi_team2.isnull().sum()

idSchool                0
siruesCode              0
shortName               0
county                  0
locality                0
studyFormationsCount    0
studentsCount           0
personnelCount          0
generalState            0
buildingMaterial        0
roofMaterial            0
roofCondition           0
joineryCondition        0
windowJoinery           0
doorJoinery             0
cod_SIIIR               0
jud_loc                 0
COD UNIC CANDIDAT       0
SEX                     0
MEDIU                   0
MEDIA                   0
MEDIA V-VIII            0
SIRUTA                  0
LHDI2018                0
IDUL2018quintile        0
hcapmat18               0
hconectint18            0
hrsm18                  0
POPD2018                0
SIRINF                  0
dtype: int64

In [99]:
#verific daca exista valori duplicate
print(school_buildings_siiir_en_lhdi_team2.duplicated().sum())

0


In [100]:
#export tabelul obtinut intr-un csv pentru verificare
output_file_path_final = 'school_buildings_siiir_en_lhdi_team2.csv'
school_buildings_siiir_en_lhdi_team2.to_csv(output_file_path_final, index=False)

In [101]:
#obtain the parameters from the pyodbc connection string
params = urllib.parse.quote_plus(connection_string)
#create the engine
sqlalchemy_engine = db.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [102]:
sqlalchemy_engine

Engine(mssql+pyodbc:///?odbc_connect=DRIVER%3D%7BODBC+Driver+18+for+SQL+Server%7D%3BSERVER%3Dtcp%3Aaid-2023-srv.database.windows.net%2C1433%3BDATABASE%3DAID2023%3BUID%3Daid2023adminlog%3BPWD%3D%40Aidr2023sql%3Bcharset%3Dutf8)

In [103]:
#verific coloanele
school_buildings_siiir_en_lhdi_team2.columns

Index(['idSchool', 'siruesCode', 'shortName', 'county', 'locality',
       'studyFormationsCount', 'studentsCount', 'personnelCount',
       'generalState', 'buildingMaterial', 'roofMaterial', 'roofCondition',
       'joineryCondition', 'windowJoinery', 'doorJoinery', 'cod_SIIIR',
       'jud_loc', 'COD UNIC CANDIDAT', 'SEX', 'MEDIU', 'MEDIA', 'MEDIA V-VIII',
       'SIRUTA', 'LHDI2018', 'IDUL2018quintile', 'hcapmat18', 'hconectint18',
       'hrsm18', 'POPD2018', 'SIRINF'],
      dtype='object')

In [104]:
# Get the list of installed drivers
drivers = [driver for driver in pyodbc.drivers()]

# Print the list of drivers
print("Installed ODBC Drivers:")
for driver in drivers:
    print(driver)

Installed ODBC Drivers:
SQL Server
MySQL ODBC 8.0 ANSI Driver
MySQL ODBC 8.0 Unicode Driver
Microsoft Access Driver (*.mdb, *.accdb)
Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)
Microsoft Access Text Driver (*.txt, *.csv)
ODBC Driver 18 for SQL Server


In [105]:
#verific tabelul final
school_buildings_siiir_en_lhdi_team2.head()

,idSchool,siruesCode,shortName,county,locality,studyFormationsCount,studentsCount,personnelCount,generalState,buildingMaterial,...,MEDIA,MEDIA V-VIII,SIRUTA,LHDI2018,IDUL2018quintile,hcapmat18,hconectint18,hrsm18,POPD2018,SIRINF
0,11266917,120437849,"COLEGIUL NATIONAL ""EMIL RACOVITA"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,36,1071,88,1,CARAMIDA,...,9.85,9.89,54975,80.1,5,80.5,98.5,26.9,32367,54984
1,11266917,120437849,"COLEGIUL NATIONAL ""EMIL RACOVITA"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,36,1071,88,1,CARAMIDA,...,9.67,9.67,54975,80.1,5,80.5,98.5,26.9,32367,54984
2,11266917,120437849,"COLEGIUL NATIONAL ""EMIL RACOVITA"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,36,1071,88,1,CARAMIDA,...,9.40,9.86,54975,80.1,5,80.5,98.5,26.9,32367,54984
3,11266917,120437849,"COLEGIUL NATIONAL ""EMIL RACOVITA"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,36,1071,88,1,CARAMIDA,...,9.00,9.21,54975,80.1,5,80.5,98.5,26.9,32367,54984
4,11266917,120437849,"COLEGIUL NATIONAL ""EMIL RACOVITA"" CLUJ-NAPOCA",CLUJ,CLUJ-NAPOCA,36,1071,88,1,CARAMIDA,...,9.95,9.85,54975,80.1,5,80.5,98.5,26.9,32367,54984


In [106]:
#export tabelul final in baza de date Azure
school_buildings_siiir_en_lhdi_team2.to_sql("school_buildings_siiir_en_lhdi_team2", sqlalchemy_engine, if_exists='replace', index=False) 

45